In [1]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [3]:
from openai import OpenAI

client = OpenAI(
    api_key="sk-al8LY2KP9JUls1HdCrdrhA2GofI2OztrBR3w1W6pErT3BlbkFJsLFqgNeJKhTgVR3VIBIHVaZ1qvi5FNsS_cBG7rTGIA"
)

assistant = client.beta.assistants.create(
    name="외환 전문가 어시스턴트",
    instructions="너는 외환 분야 전문가야, 특히 '원/달러' 분야 전문인 경제 전문가야. 사용자가 외환 관련 리포트를 전달하면, 아래와 같은 정보들을 사용자에게 전달해야 해: 1. 오늘 원달러가 오를지 내려갈지와 이유 2. 오늘 원/달러 가격이 얼마나 변동이 있을지와 이유 3. 오늘 원달러 종가와 그에 대한 분석. 4. 일주일간 원달러가 오를지 내려갈지와 이유 5. 일주일간 원/달러 가격이 얼마나 변동이 있을지와 이유 6. 일주일간 원달러 종가와 그에 대한 분석. 7. 한달간 원달러가 오를지 내려갈지와 이유 8. 한달간 원/달러 가격이 얼마나 변동이 있을지와 이유 9. 한달간 원달러 종가와 그에 대한 분석.",
    model="gpt-4o",
    tools=[{"type": "file_search"}],
)

# 벡터 스토어 생성
vector_store = client.beta.vector_stores.create(name="외환 리포트 벡터 스토어")

# 파일 경로 설정
file_paths = ["/Users/yang-jin-an/Downloads/report/Weekly240923.pdf"]
file_streams = [open(path, "rb") for path in file_paths]

# 파일을 업로드하고 벡터 스토어에 추가
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=file_streams
)

# 업로드 결과 확인
print(file_batch.status)
print(file_batch.file_counts)

assistant = client.beta.assistants.update(
    assistant_id=assistant.id,
    tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

# 스레드 생성
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "안녕하세요, 오늘 원/달러 환율 전망에 대해 알려주세요.",
        }
    ]
)

# Run 생성 및 상태 확인
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

# 메시지 출력
messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))
assistant_reply = messages[-1]["content"][0]["text"]["value"]
print(assistant_reply)
print(messages)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)
[Message(id='msg_GkAeoCh6w1vP94IJbYSomYOt', assistant_id='asst_HTjlFRKGpc3QFk76kx2T4TBS', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=138, file_citation=FileCitation(file_id='file-RBnEkJJ8BP6n6ZNVrBIjaoq4'), start_index=126, text='【6:0†source】', type='file_citation'), FileCitationAnnotation(end_index=275, file_citation=FileCitation(file_id='file-RBnEkJJ8BP6n6ZNVrBIjaoq4'), start_index=263, text='【6:0†source】', type='file_citation'), FileCitationAnnotation(end_index=390, file_citation=FileCitation(file_id='file-RBnEkJJ8BP6n6ZNVrBIjaoq4'), start_index=378, text='【6:0†source】', type='file_citation')], value='### 오늘 원/달러 환율 전망\n\n1. **오늘 원/달러가 오를지 내려갈지와 이유**:\n   오늘 원/달러 환율은 소폭 상승할 것으로 전망됩니다. 이에 대한 주요 이유는 강달러 추세와 한국 외환 시장의 불확실성 증가 때문입니다【6:0†source】.\n\n2. **오늘 원/달러 가격이 얼마나 변동이 있을지와 이유**:\n   변동폭은 시장의 마지막 동향 기준으로 5~10원 내외로 예측됩니다. 이는 글로벌 경제

In [ ]:
# 메시지 객체의 value 필드를 추출하는 함수
def extract_value_from_message(messages):
    # message의 content[0]의 text 필드에서 value를 추출
    if message.content and len(message.content) > 0:
        return message.content[0].text.value
    return None

# 메시지에서 value 추출
message = messages[-1]  # 가장 마지막 메시지를 사용
value = extract_value_from_message(message)

# 추출된 value 출력
if value:
    print(value)
else:
    print("value를 추출할 수 없습니다.")

In [5]:
messages = "[Message(id='msg_GkAeoCh6w1vP94IJbYSomYOt', assistant_id='asst_HTjlFRKGpc3QFk76kx2T4TBS', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=138, file_citation=FileCitation(file_id='file-RBnEkJJ8BP6n6ZNVrBIjaoq4'), start_index=126, text='【6:0†source】', type='file_citation'), FileCitationAnnotation(end_index=275, file_citation=FileCitation(file_id='file-RBnEkJJ8BP6n6ZNVrBIjaoq4'), start_index=263, text='【6:0†source】', type='file_citation'), FileCitationAnnotation(end_index=390, file_citation=FileCitation(file_id='file-RBnEkJJ8BP6n6ZNVrBIjaoq4'), start_index=378, text='【6:0†source】', type='file_citation')], value='### 오늘 원/달러 환율 전망\n\n1. **오늘 원/달러가 오를지 내려갈지와 이유**:\n   오늘 원/달러 환율은 소폭 상승할 것으로 전망됩니다. 이에 대한 주요 이유는 강달러 추세와 한국 외환 시장의 불확실성 증가 때문입니다【6:0†source】.\n\n2. **오늘 원/달러 가격이 얼마나 변동이 있을지와 이유**:\n   변동폭은 시장의 마지막 동향 기준으로 5~10원 내외로 예측됩니다. 이는 글로벌 경제 지표 발표와 외환 거래량의 변화 등 여러 요인에 따라 결정됩니다【6:0†source】.\n\n3. **오늘 원달러 종가와 그에 대한 분석**:\n   오늘 종가는 1,190원대 초반으로 예상됩니다. 이는 미 달러의 강세와 한국 경제 지표의 부진이 영향을 미칠 것으로 보입니다【6:0†source】.\n\n### 추가 정보\n\n**일주일, 한달 간 전망 및 분석**도 필요하시면 언제든지 요청해주세요. 더 풍부한 데이터와 분석으로 지원하겠습니다.'), type='text')], created_at=1727156123, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_hSQY1ncnHMkIzDoa6g1HFOyB', status=None, thread_id='thread_T0uI2J3rqcjumWGzbzY267Z7')]"


In [7]:
from openai import OpenAI

# Initialize the OpenAI client using an environment variable for the API key

client = OpenAI(
    api_key="sk-al8LY2KP9JUls1HdCrdrhA2GofI2OztrBR3w1W6pErT3BlbkFJsLFqgNeJKhTgVR3VIBIHVaZ1qvi5FNsS_cBG7rTGIA"
)
# Create the assistant with the system prompt included in 'instructions'
assistant = client.beta.assistants.create(
    name="외환 전문가 어시스턴트",
    instructions="""
        너는 외환 분야 전문가야, 특히 '원/달러' 분야 전문인 경제 전문가야. 사용자가 외환 관련 리포트를 전달하면, 
        아래와 같은 정보들을 사용자에게 전달해야 해: 
        1. 오늘 원달러가 오를지 내려갈지와 이유 
        2. 오늘 원/달러 가격이 얼마나 변동이 있을지와 이유 
        3. 오늘 원달러 종가와 그에 대한 분석. 
        4. 일주일간 원달러가 오를지 내려갈지와 이유 
        5. 일주일간 원/달러 가격이 얼마나 변동이 있을지와 이유 
        6. 일주일간 원달러 종가와 그에 대한 분석. 
        7. 한달간 원달러가 오를지 내려갈지와 이유 
        8. 한달간 원/달러 가격이 얼마나 변동이 있을지와 이유 
        9. 한달간 원달러 종가와 그에 대한 분석.
        """,
    model="gpt-4o",
    tools=[{"type": "file_search"}],
)

# Create the vector store
vector_store = client.beta.vector_stores.create(name="외환 리포트 벡터 스토어")

# File paths for multiple files
file_paths = [
    "/Users/yang-jin-an/Downloads/report/Weekly240923.pdf",
    "/Users/yang-jin-an/Downloads/report/semi2024.pdf",
    "/Users/yang-jin-an/Downloads/report/Monthly2409.pdf"
]

# Open file streams
file_streams = [open(path, "rb") for path in file_paths]

# Upload the files and add them to the vector store
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=file_streams
)

# Check the upload status
print(file_batch.status)
print(file_batch.file_counts)

# Update the assistant to use the new vector store
assistant = client.beta.assistants.update(
    assistant_id=assistant.id,
    tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

# Create a thread with the user's message
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "안녕하세요, 오늘 원/달러 환율 전망에 대해 알려주세요.",
        }
    ]
)

# Create a run and wait for it to complete
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

# Retrieve and print the assistant's reply
messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))
# assistant_reply = messages[-1]["content"][0]["text"]["value"]
# print(assistant_reply)
print(messages)

# Close the file streams
for file_stream in file_streams:
    file_stream.close()


completed
FileCounts(cancelled=0, completed=3, failed=0, in_progress=0, total=3)
[Message(id='msg_0DAWAsJ2sPDbfubUg2uu0UQh', assistant_id='asst_sVrkfG6SnVgLqKRYI8TWzgpQ', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=293, file_citation=FileCitation(file_id='file-rRN13wCfjJ9pxxEZLufsYbUJ'), start_index=271, text='【4:0†Weekly240923.pdf】', type='file_citation'), FileCitationAnnotation(end_index=444, file_citation=FileCitation(file_id='file-rRN13wCfjJ9pxxEZLufsYbUJ'), start_index=422, text='【4:0†Weekly240923.pdf】', type='file_citation'), FileCitationAnnotation(end_index=634, file_citation=FileCitation(file_id='file-rRN13wCfjJ9pxxEZLufsYbUJ'), start_index=612, text='【4:0†Weekly240923.pdf】', type='file_citation'), FileCitationAnnotation(end_index=800, file_citation=FileCitation(file_id='file-oVjnXNWFkamosq4Ok8a17UgR'), start_index=779, text='【6:2†Monthly2409.pdf】', type='file_citation'), FileCitationAnnotation(end_index=9

In [5]:
from openai import OpenAI
import os

# 환경 변수에서 API 키를 가져옵니다 (API 키는 코드에 직접 포함하지 않는 것이 좋습니다)
client = OpenAI(
    api_key="sk-al8LY2KP9JUls1HdCrdrhA2GofI2OztrBR3w1W6pErT3BlbkFJsLFqgNeJKhTgVR3VIBIHVaZ1qvi5FNsS_cBG7rTGIA"
)

# 코드 인터프리터를 tools에 추가합니다
assistant = client.beta.assistants.create(
    name="외환 전문가 어시스턴트",
    instructions="""
        너는 외환 분야 전문가야, 특히 '원/달러' 분야 전문인 경제 전문가야. 사용자가 외환 관련 리포트를 전달하면, 
        아래와 같은 정보들을 사용자에게 전달해야 해: 
        1. 오늘 원달러가 오를지 내려갈지와 이유 
        2. 오늘 원/달러 가격이 얼마나 변동이 있을지와 이유 
        3. 오늘 원달러 종가와 그에 대한 분석. 
        4. 일주일간 원달러가 오를지 내려갈지와 이유 
        5. 일주일간 원/달러 가격이 얼마나 변동이 있을지와 이유 
        6. 일주일간 원달러 종가와 그에 대한 분석. 
        7. 한달간 원달러가 오를지 내려갈지와 이유 
        8. 한달간 원/달러 가격이 얼마나 변동이 있을지와 이유 
        9. 한달간 원달러 종가와 그에 대한 분석.
    """,
    model="gpt-4o",
    tools=[
        {"type": "file_search"},
        {"type": "code_interpreter"}
    ],
)

# 벡터 스토어 생성
vector_store = client.beta.vector_stores.create(name="외환 리포트 벡터 스토어")

# 기존 PDF 파일 경로와 추가할 PDF 파일 경로를 합칩니다
pdf_file_paths = [
    "/Users/yang-jin-an/Downloads/report/Weekly240923.pdf",
    "/Users/yang-jin-an/Downloads/report/semi2024.pdf",
    "/Users/yang-jin-an/Downloads/report/Monthly2409.pdf",
    "/Users/yang-jin-an/Downloads/report/Daily240923.pdf",
    "/Users/yang-jin-an/Downloads/report/Daily240924.pdf",
    "/Users/yang-jin-an/Downloads/report/Daily240925.pdf"
]

# PDF 파일 스트림 열기
pdf_file_streams = [open(path, "rb") for path in pdf_file_paths]

# PDF 파일을 업로드하고 벡터 스토어에 추가
pdf_file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=pdf_file_streams
)

# PDF 파일 스트림 닫기
for file_stream in pdf_file_streams:
    file_stream.close()

# CSV 파일 경로
csv_file_paths = [
    "/Users/yang-jin-an/Downloads/rate/DFII10.csv",
    "/Users/yang-jin-an/Downloads/rate/saudi_oil.csv",
    "/Users/yang-jin-an/Downloads/rate/WTI_oil.csv"
]

# CSV 파일 스트림 열기
csv_file_streams = [open(path, "rb") for path in csv_file_paths]

# CSV 파일 업로드 및 파일 ID 수집
csv_file_ids = []
for file_stream in csv_file_streams:
    csv_file = client.files.create(file=file_stream)
    csv_file_ids.append(csv_file.id)
    file_stream.close()

# 어시스턴트에 새로운 벡터 스토어와 CSV 파일을 추가
assistant = client.beta.assistants.update(
    assistant_id=assistant.id,
    tool_resources={
        "file_search": {"vector_store_ids": [vector_store.id]},
        "code_interpreter": {"file_ids": csv_file_ids},
    },
)

# 사용자 메시지로 스레드 생성
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "안녕하세요, 오늘 원/달러 환율 전망에 대해 알려주세요.",
        }
    ]
)

# 실행 생성 및 완료 대기
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

# 어시스턴트의 응답 가져오기 및 출력
messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))
print(messages)


TypeError: create() missing 1 required keyword-only argument: 'purpose'

In [7]:
from openai import OpenAI
import os

# 환경 변수에서 API 키를 가져옵니다
client = OpenAI(
    api_key="sk-al8LY2KP9JUls1HdCrdrhA2GofI2OztrBR3w1W6pErT3BlbkFJsLFqgNeJKhTgVR3VIBIHVaZ1qvi5FNsS_cBG7rTGIA"
)


# 코드 인터프리터를 tools에 추가합니다
assistant = client.beta.assistants.create(
    name="외환 전문가 어시스턴트",
    instructions="""
        너는 외환 분야 전문가야, 특히 '원/달러' 분야 전문인 경제 전문가야. 사용자가 외환 관련 리포트를 전달하면, 
        아래와 같은 정보들을 사용자에게 전달해야 해: 
        1. 오늘 원달러가 오를지 내려갈지와 이유 
        2. 오늘 원/달러 가격이 얼마나 변동이 있을지와 이유 
        3. 오늘 원달러 종가와 그에 대한 분석. 
        4. 일주일간 원달러가 오를지 내려갈지와 이유 
        5. 일주일간 원/달러 가격이 얼마나 변동이 있을지와 이유 
        6. 일주일간 원달러 종가와 그에 대한 분석. 
        7. 한달간 원달러가 오를지 내려갈지와 이유 
        8. 한달간 원/달러 가격이 얼마나 변동이 있을지와 이유 
        9. 한달간 원달러 종가와 그에 대한 분석.
    """,
    model="gpt-4o",
    tools=[
        {"type": "file_search"},
        {"type": "code_interpreter"}
    ],
)

# 벡터 스토어 생성
vector_store = client.beta.vector_stores.create(name="외환 리포트 벡터 스토어")

# 기존 PDF 파일 경로와 추가할 PDF 파일 경로를 합칩니다
pdf_file_paths = [
    "/Users/yang-jin-an/Downloads/report/Weekly240923.pdf",
    "/Users/yang-jin-an/Downloads/report/semi2024.pdf",
    "/Users/yang-jin-an/Downloads/report/Monthly2409.pdf",
    "/Users/yang-jin-an/Downloads/report/Daily240923.pdf",
    "/Users/yang-jin-an/Downloads/report/Daily240924.pdf",
    "/Users/yang-jin-an/Downloads/report/Daily240925.pdf"
]

# PDF 파일 스트림 열기
pdf_file_streams = [open(path, "rb") for path in pdf_file_paths]

# PDF 파일을 업로드하고 벡터 스토어에 추가
pdf_file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=pdf_file_streams
)

# PDF 파일 스트림 닫기
for file_stream in pdf_file_streams:
    file_stream.close()

# CSV 파일 경로
csv_file_paths = [
    "/Users/yang-jin-an/Downloads/rate/DFII10.csv",
    "/Users/yang-jin-an/Downloads/rate/saudi_oil.csv",
    "/Users/yang-jin-an/Downloads/rate/WTI_oil.csv"
]

# CSV 파일 스트림 열기
csv_file_streams = [open(path, "rb") for path in csv_file_paths]

# 코드 인터프리터 세션 생성
code_interpreter_session = client.beta.code_interpreter.sessions.create()

# CSV 파일 업로드 및 파일 ID 수집
csv_file_ids = []
for file_stream in csv_file_streams:
    csv_file = client.beta.code_interpreter.sessions.files.upload(
        session_id=code_interpreter_session.id,
        file=file_stream
    )
    csv_file_ids.append(csv_file.id)
    file_stream.close()

# 어시스턴트에 새로운 벡터 스토어와 코드 인터프리터 세션을 추가
assistant = client.beta.assistants.update(
    assistant_id=assistant.id,
    tool_resources={
        "file_search": {"vector_store_ids": [vector_store.id]},
        "code_interpreter": {"session_ids": [code_interpreter_session.id]},
    },
)

# 사용자 메시지로 스레드 생성
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "안녕하세요, 오늘 원/달러 환율 전망에 대해 알려주세요.",
        }
    ]
)

# 실행 생성 및 완료 대기
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

# 어시스턴트의 응답 가져오기 및 출력
messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))
print(messages)


AttributeError: 'Beta' object has no attribute 'code_interpreter'

In [9]:
from openai import OpenAI
import os

# 환경 변수에서 API 키를 가져옵니다
client = OpenAI(
    api_key="sk-al8LY2KP9JUls1HdCrdrhA2GofI2OztrBR3w1W6pErT3BlbkFJsLFqgNeJKhTgVR3VIBIHVaZ1qvi5FNsS_cBG7rTGIA"
)

# 코드 인터프리터를 tools에 추가합니다
assistant = client.beta.assistants.create(
    name="외환 전문가 어시스턴트",
    instructions="""
        너는 외환 분야 전문가야, 특히 '원/달러' 분야 전문인 경제 전문가야. 사용자가 외환 관련 리포트를 전달하면, 
        아래와 같은 정보들을 사용자에게 전달해야 해: 
        1. 오늘 원달러가 오를지 내려갈지와 이유 
        2. 오늘 원/달러 가격이 얼마나 변동이 있을지와 이유 
        3. 오늘 원달러 종가와 그에 대한 분석. 
        4. 일주일간 원달러가 오를지 내려갈지와 이유 
        5. 일주일간 원/달러 가격이 얼마나 변동이 있을지와 이유 
        6. 일주일간 원달러 종가와 그에 대한 분석. 
        7. 한달간 원달러가 오를지 내려갈지와 이유 
        8. 한달간 원/달러 가격이 얼마나 변동이 있을지와 이유 
        9. 한달간 원달러 종가와 그에 대한 분석.
    """,
    model="gpt-4o",
    tools=[
        {"type": "file_search"},
        {"type": "code_interpreter"}
    ],
)

# 벡터 스토어 생성
vector_store = client.beta.vector_stores.create(name="외환 리포트 벡터 스토어")

# PDF 파일 경로들
pdf_file_paths = [
    "/Users/yang-jin-an/Downloads/report/Weekly240923.pdf",
    "/Users/yang-jin-an/Downloads/report/semi2024.pdf",
    "/Users/yang-jin-an/Downloads/report/Monthly2409.pdf",
    "/Users/yang-jin-an/Downloads/report/Daily240923.pdf",
    "/Users/yang-jin-an/Downloads/report/Daily240924.pdf",
    "/Users/yang-jin-an/Downloads/report/Daily240925.pdf"
]

# PDF 파일 스트림 열기
pdf_file_streams = [open(path, "rb") for path in pdf_file_paths]

# PDF 파일을 업로드하고 벡터 스토어에 추가
pdf_file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=pdf_file_streams
)

# PDF 파일 스트림 닫기
for file_stream in pdf_file_streams:
    file_stream.close()

# CSV 파일 경로들
csv_file_paths = [
    "/Users/yang-jin-an/Downloads/rate/DFII10.csv",
    "/Users/yang-jin-an/Downloads/rate/saudi_oil.csv",
    "/Users/yang-jin-an/Downloads/rate/WTI_oil.csv"
]

# CSV 파일 스트림 열기
csv_file_streams = [open(path, "rb") for path in csv_file_paths]

# CSV 파일 업로드 및 파일 ID 수집
csv_file_ids = []
for file_stream in csv_file_streams:
    csv_file = client.files.create(
        file=file_stream,
        purpose='assistants'  # 'purpose' 인자를 추가합니다
    )
    csv_file_ids.append(csv_file.id)
    file_stream.close()

# 어시스턴트에 새로운 벡터 스토어와 CSV 파일을 추가
assistant = client.beta.assistants.update(
    assistant_id=assistant.id,
    tool_resources={
        "file_search": {"vector_store_ids": [vector_store.id]},
        "code_interpreter": {"file_ids": csv_file_ids},  # 파일 ID를 전달합니다
    },
)

# 사용자 메시지로 스레드 생성
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "안녕하세요, 오늘 원/달러 환율 전망에 대해 알려주세요.",
        }
    ]
)

# 실행 생성 및 완료 대기
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

# 어시스턴트의 응답 가져오기 및 출력
messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))
print(messages)

# CSV 파일 스트림 닫기
for file_stream in csv_file_streams:
    file_stream.close()
    
{
    내일원달러 : "".
    변동 : "",
    일주알뒤 원달러: "",
    주변동
    월 :
    월변동 :
    분석
    동향 :
    한달치 예측 환율 : 
    이 리포트 근거 :
}

"ai 외환 동향 예측"

[Message(id='msg_kJQfKiWzE3lWXVLg8WQRkXwR', assistant_id='asst_xGz2HtTIg9Azo2gXgIcyqCVs', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=306, file_citation=FileCitation(file_id='file-XJb5IYdgzA1afSlFrzLuUZ6r'), start_index=284, text='【12:2†Daily240923.pdf】', type='file_citation'), FileCitationAnnotation(end_index=392, file_citation=FileCitation(file_id='file-TaKohKys8pYZZyqBvjsWgtLP'), start_index=370, text='【12:3†Daily240924.pdf】', type='file_citation'), FileCitationAnnotation(end_index=600, file_citation=FileCitation(file_id='file-XJb5IYdgzA1afSlFrzLuUZ6r'), start_index=578, text='【12:2†Daily240923.pdf】', type='file_citation'), FileCitationAnnotation(end_index=823, file_citation=FileCitation(file_id='file-XJb5IYdgzA1afSlFrzLuUZ6r'), start_index=801, text='【12:2†Daily240923.pdf】', type='file_citation')], value='오늘 원달러 환율 전망과 관련된 정보를 제공할 수 있도록 분석을 해보겠습니다. 먼저 필요한 정보들을 요약하고, 그에 따른 전망을 제시하겠습니다:\n\n### 1. 오늘 원달러가 오를지 내려갈

In [10]:
def extract_values_from_messages(messages):
    values = []
    for message in messages:
        for content_block in message.content:
            if hasattr(content_block, 'text') and hasattr(content_block.text, 'value'):
                values.append(content_block.text.value)
    return values


In [11]:
extracted_values = extract_values_from_messages(messages)
for value in extracted_values:
    print(value)

오늘 원달러 환율 전망과 관련된 정보를 제공할 수 있도록 분석을 해보겠습니다. 먼저 필요한 정보들을 요약하고, 그에 따른 전망을 제시하겠습니다:

### 1. 오늘 원달러가 오를지 내려갈지와 이유
오늘 원달러 환율은 현재 글로벌 외환 시장의 동향, 미국 달러 지수(DXY)의 움직임, 그리고 주요 경제 지표나 뉴스 이벤트에 따라 영향을 받을 것입니다. 최근의 경제 보고서에 따르면:

- 글로벌 달러 지수(DXY)의 움직임이 상방향으로 나타나고 있으며, 이는 원/달러 환율이 상승할 가능성이 높음을 시사합니다【12:2†Daily240923.pdf】.
- 글로벌 시장에서 위험 회피 성향이 증가하고 있으며, 이는 안전 자산으로서 달러의 수요를 증가시킬 수 있습니다【12:3†Daily240924.pdf】.

따라서, 이러한 요인들을 종합하면 오늘 원달러 환율이 상승할 가능성이 있다고 분석됩니다.

### 2. 오늘 원/달러 가격이 얼마나 변동이 있을지와 이유
오늘 원/달러 환율의 변동 폭은 다음과 같은 요인들에 의해 결정될 수 있습니다:

- 마지막 보고서에 따르면 원달러 변동 범위는 1328에서 1337 사이로 예상되고 있습니다【12:2†Daily240923.pdf】.
- 주요 글로벌 경제 지표 발표 및 미국 연방준비제도의 정책 결정 등이 변동성을 높일 수 있습니다.

그러므로, 오늘 원달러 환율은 약 ±5원 내외로 변동될 가능성이 높습니다.

### 3. 오늘 원달러 종가와 그에 대한 분석
오늘의 원달러 종가 예측은 1335원 수준으로 볼 수 있습니다. 이는 글로벌 달러 강세와 국내 경제지표의 영향을 고려한 결과입니다【12:2†Daily240923.pdf】.

이 분석은 당일의 경제 이벤트나 글로벌 금융 시장의 영향을 크게 받을 수 있습니다.

오늘의 원달러 환율 전망을 종합하여 봤을 때, 환율이 상승할 가능성이 높고, 변동 폭은 약 ±5원, 종가는 1335원 수준으로 예상됩니다. 이 정보가 투자나 경제 활동에 도움이 되길 바랍니다.


In [20]:
from openai import OpenAI
import os

# 환경 변수에서 API 키를 가져옵니다
client = OpenAI(
    api_key="sk-al8LY2KP9JUls1HdCrdrhA2GofI2OztrBR3w1W6pErT3BlbkFJsLFqgNeJKhTgVR3VIBIHVaZ1qvi5FNsS_cBG7rTGIA"
)

# 코드 인터프리터를 tools에 추가합니다
assistant = client.beta.assistants.create(
    name="외환 전문가 어시스턴트",
    instructions="""
        너는 외환 분야 전문가야, 특히 '원/달러' 분야 전문인 경제 전문가야. 
        너에게는 일간, 주간, 월간, 연간 외환 동향을 분석한 리포트가 PDF 파일 형식으로 제공될 거고,
        미국 금리, 유가, 원/달러, DXY의 추세를 기록한 시계열 데이터가 CSV 파일 형식으로 제공될 거야.
        
        먼저 PDF 파일들에 대해서 설명할게:
        1. **Daily240923.pdf**: 2024/09/23의 외환 동향 리포트.
        2. **Daily240924.pdf**: 2024/09/24의 외환 동향 리포트.
        3. **Daily240925.pdf**: 2024/09/25의 외환 동향 리포트.
        4. **Weekly240923.pdf**: 2024/09/23까지의 주간 외환 동향 리포트.
        5. **Monthly2409.pdf**: 2024년 9월 한 달간의 외환 동향 리포트.
        6. **semi2024.pdf**: 2024년 상반기 외환 동향 리포트.
        
        다음은 CSV 파일들에 대한 설명이야:
        
        1. **DFII10.csv** (2019-2024년 미국 실질 금리 데이터):
            - **date**: 거래 날짜 (YYYY/M/DD 형식).
            - **DFII10**: 해당 날짜의 미국 실질 금리(TIPS) 값.
        
        2. **dxy.csv** (2015-2024년 미국 달러 지수 데이터):
            - **date**: 거래 날짜 (YYYY/M/DD 형식).
            - **end_price**: 장 마감 시 미국 달러 지수 값.
            - **start_price**: 장 시작 시 미국 달러 지수 값.
            - **high_price**: 하루 중 가장 높았던 미국 달러 지수 값.
            - **low_price**: 하루 중 가장 낮았던 미국 달러 지수 값.
            - **fluctuating**: 직전 값과 비교한 변동 비율 (0.01% 단위로 반영).
        
        3. **WTI_oil.csv** (2015-2024년 저유황 경질유(WTI) 유가 데이터):
            - **date**: 거래 날짜 (YYYY/M/DD 형식).
            - **end_price**: 장 마감 
            시 WTI 유가 (1배럴당).
            - **start_price**: 장 시작 시 WTI 유가 (1배럴당).
            - **high_price**: 하루 중 가장 높았던 WTI 유가 (1배럴당).
            - **low_price**: 하루 중 가장 낮았던 WTI 유가 (1배럴당).
            - **fluctuating**: 직전 값과 비교한 변동 비율 (0.01% 단위로 반영).
        
        4. **saudi_oil.csv** (2015-2024년 사우디아라비아 유가 데이터):
            - **date**: 거래 날짜 (YYYY/M/DD 형식).
            - **end_price**: 장 마감 시 사우디아라비아 유가 (1배럴당).
            - **start_price**: 장 시작 시 사우디아라비아 유가 (1배럴당).
            - **high_price**: 하루 중 가장 높았던 사우디아라비아 유가 (1배럴당).
            - **low_price**: 하루 중 가장 낮았던 사우디아라비아 유가 (1배럴당).
            - **fluctuating**: 직전 값과 비교한 변동 비율 (0.01% 단위로 반영).
        
        5. **exchange_rate.csv** (2010-2024년 원/달러 환율 데이터):
            - **date**: 거래 날짜 (YYYY/M/DD 형식).
            - **base_rate**: 해당 날짜의 매매 기준율 종가.

        이 CSV와 PDF 파일들에 기반해 다음과 같은 분석을 제공해야 해. 단 CSV 파일을 읽어서 예측모델을 모델링하는 것은 금지야. 
        CSV 파일은 원/달러 환율을 예측하는데만 사용해야해 이 부분은 반드시 지켜야해.
        
        다음은 분석 프로세스야 :
        - PDF를 열람 후 분석
        - CSV를 열람 후 분석
        - 결과 값 응답 
        
        너는 사용자가 요청한 정보를 항상 다음 양식에 맞춰 반환해야 해(토큰 제한이 있으니까 반드시 응답양식과 일치하는 답변만 해야해):
        
        {
            "2024/09/25": "원/달러 종가",
            "2024/09/26": "원/달러 종가",
            "주간" : "앞으로 한주간 원달러 종가",
            "이유" : "이렇게 분석한 이유"
        }
        
        다음은 응답양식 예시야 : 
        
        {
            "2024/09/25": "1330.20",
            "2024/09/26": "1329.10",
            "주간" : " 2024/09/27 : 1328.10, 2024/09/28 : 1328.10, 2024/09/29 : 1328.10, 2024/09/30 : 1327.10, 2024/10/01 : 1328.10, 2024/10/02 : 1326.20, 2024/10/03 : 1327.10"
            "이유" : "2024/09/24와 2024/09/25의 일간 리포트를 분석한 결과...."
        }

    """,
    model="gpt-4o",
    tools=[
        {"type": "file_search"},
        {"type": "code_interpreter"}
    ],
)

# 벡터 스토어 생성
vector_store = client.beta.vector_stores.create(name="외환 리포트 벡터 스토어")

# PDF 파일 경로들
pdf_file_paths = [
    "/Users/yang-jin-an/Downloads/report/Daily240923.pdf",
    "/Users/yang-jin-an/Downloads/report/Daily240924.pdf",
    "/Users/yang-jin-an/Downloads/report/Daily240925.pdf",
    "/Users/yang-jin-an/Downloads/report/Weekly240923.pdf",
    "/Users/yang-jin-an/Downloads/report/Monthly2409.pdf",
    "/Users/yang-jin-an/Downloads/report/semi2024.pdf",
]

# PDF 파일 스트림 열기
pdf_file_streams = [open(path, "rb") for path in pdf_file_paths]

# PDF 파일을 업로드하고 벡터 스토어에 추가
pdf_file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=pdf_file_streams
)

# PDF 파일 스트림 닫기
for file_stream in pdf_file_streams:
    file_stream.close()

# CSV 파일 경로들
csv_file_paths = [
    "/Users/yang-jin-an/Downloads/rate/DFII10.csv",
    "/Users/yang-jin-an/Downloads/rate/saudi_oil.csv",
    "/Users/yang-jin-an/Downloads/rate/WTI_oil.csv",
    "/Users/yang-jin-an/Downloads/rate/exchange_rate.csv"
]

# CSV 파일 스트림 열기
csv_file_streams = [open(path, "rb") for path in csv_file_paths]

# CSV 파일 업로드 및 파일 ID 수집
csv_file_ids = []
for file_stream in csv_file_streams:
    csv_file = client.files.create(
        file=file_stream,
        purpose='assistants'  # 'purpose' 인자를 추가합니다
    )
    csv_file_ids.append(csv_file.id)
    file_stream.close()

# 어시스턴트에 새로운 벡터 스토어와 CSV 파일을 추가
assistant = client.beta.assistants.update(
    assistant_id=assistant.id,
    tool_resources={
        "file_search": {"vector_store_ids": [vector_store.id]},
        "code_interpreter": {"file_ids": csv_file_ids},  # 파일 ID를 전달합니다
    },
)

# 사용자 메시지로 스레드 생성
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "2024/09/25 2024/09/26 원/달러 환율 종가와 변동범위 그리고 앞으로 일주일간 원/달러 환율 종가를 알려줘 오늘은 2024/09/25 고 원달러 환율을 말할 필요없어 미래에 대한걸 추론해서 말해.",
        }
    ]
)

# 실행 생성 및 완료 대기
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

# 어시스턴트의 응답 가져오기 및 출력
messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))
print(messages)

# CSV 파일 스트림 닫기
for file_stream in csv_file_streams:
    file_stream.close()
    
extracted_values = extract_values_from_messages(messages)
for value in extracted_values:
    print(value)

[Message(id='msg_r8VX4AvYJ1E34yB7EU1rNMUG', assistant_id='asst_cYWUgCR2gn2Jyd0Bn1hK5z4I', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='데이터에서 `base_rate` 열에 포함된 값이 문자열 형식으로 되어 있어 숫자 형식으로 변환하는 과정에서 문제가 발생했습니다. 우선 이 문제를 해결하기 위해 불필요한 문자를 제거한 후, 숫자 형식으로 변환하겠습니다.\n\n정리된 데이터를 바탕으로 다시 예측을 시도해보겠습니다.'), type='text')], created_at=1727265173, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_o5z3ct5acwPz494vKy7ylSBH', status=None, thread_id='thread_pO2ooe3TuMhIpzHtPzov3vt7'), Message(id='msg_SuHzqrLpvKtKh8vBxjpkF9bI', assistant_id='asst_cYWUgCR2gn2Jyd0Bn1hK5z4I', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='`exchange_rate.csv` 파일에서 2024/09/25와 2024/09/26의 데이터가 누락된 것 같습니다. 최근의 데이터를 이용해 향후 일주일간의 원/달러 환율 종가를 예측하기 위해 다른 변수를 함께 고려하겠습니다.\n\n이제 모든 CSV 데이터를 최신 데이터로 결합하여 향후 일주일간의 원/달러 환율 종가를 예측하겠습니다. \n\n그러기 위해 모든 최근 데이터를 결합하고, 이를 바탕으로 향후 일주일 동안의 원

In [22]:
from openai import OpenAI
import os

# 환경 변수에서 API 키를 가져옵니다
client = OpenAI(
    api_key="sk-al8LY2KP9JUls1HdCrdrhA2GofI2OztrBR3w1W6pErT3BlbkFJsLFqgNeJKhTgVR3VIBIHVaZ1qvi5FNsS_cBG7rTGIA"
)

# 코드 인터프리터를 tools에 추가합니다
assistant = client.beta.assistants.create(
    name="외환 전문가 어시스턴트",
    instructions="""
        너는 외환 분야 전문가야, 특히 '원/달러' 분야 전문인 경제 전문가야. 
        너에게는 일간, 주간, 월간, 연간 외환 동향을 분석한 리포트가 PDF 파일 형식으로 제공될 거고,
        미국 금리, 유가, 원/달러, DXY의 추세를 기록한 시계열 데이터가 CSV 파일 형식으로 제공될 거야.
        
        먼저 PDF 파일들에 대해서 설명할게:
        1. **Daily240923.pdf**: 2024/09/23의 외환 동향 리포트.
        2. **Daily240924.pdf**: 2024/09/24의 외환 동향 리포트.
        3. **Daily240925.pdf**: 2024/09/25의 외환 동향 리포트.
        4. **Weekly240923.pdf**: 2024/09/23까지의 주간 외환 동향 리포트.
        5. **Monthly2409.pdf**: 2024년 9월 한 달간의 외환 동향 리포트.
        6. **semi2024.pdf**: 2024년 상반기 외환 동향 리포트.

        이 PDF 파일들에 기반해 다음과 같은 분석을 제공해야 해.
        
        다음은 분석 프로세스야 :
        1. PDF를 열람
        2. 결과 값 응답
        
        너는 사용자가 요청한 정보를 항상 다음 양식에 맞춰 반환해야 해(토큰 제한이 있으니까 반드시 응답양식과 일치하는 답변만 해야해):
        
        {
            "2024/09/25": "원/달러 종가",
            "2024/09/26": "원/달러 종가",
            "주간" : "앞으로 한주간 원달러 종가",
            "이유" : "이렇게 분석한 이유"
        }

    """,
    model="gpt-4o",
    tools=[
        {"type": "file_search"},
        {"type": "code_interpreter"}
    ],
)

# 벡터 스토어 생성
vector_store = client.beta.vector_stores.create(name="외환 리포트 벡터 스토어")

# PDF 파일 경로들
pdf_file_paths = [
    "/Users/yang-jin-an/Downloads/report/Daily240923.pdf",
    "/Users/yang-jin-an/Downloads/report/Daily240924.pdf",
    "/Users/yang-jin-an/Downloads/report/Daily240925.pdf",
    "/Users/yang-jin-an/Downloads/report/Weekly240923.pdf",
    "/Users/yang-jin-an/Downloads/report/Monthly2409.pdf",
    "/Users/yang-jin-an/Downloads/report/semi2024.pdf",
]

# PDF 파일 스트림 열기
pdf_file_streams = [open(path, "rb") for path in pdf_file_paths]

# PDF 파일을 업로드하고 벡터 스토어에 추가
pdf_file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=pdf_file_streams
)

# PDF 파일 스트림 닫기
for file_stream in pdf_file_streams:
    file_stream.close()


# 어시스턴트에 새로운 벡터 스토어와 CSV 파일을 추가
assistant = client.beta.assistants.update(
    assistant_id=assistant.id,
    tool_resources={
        "file_search": {"vector_store_ids": [vector_store.id]}
    },
)

# 사용자 메시지로 스레드 생성
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "2024/09/25 2024/09/26 원/달러 환율 종가와 변동범위 그리고 앞으로 일주일간 원/달러 환율 종가를 알려줘 오늘은 2024/09/25 고 원달러 환율을 말할 필요없어 미래에 대한걸 추론해서 말해.",
        }
    ]
)

# 실행 생성 및 완료 대기
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

# 어시스턴트의 응답 가져오기 및 출력
messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))
print(messages)
    
extracted_values = extract_values_from_messages(messages)
for value in extracted_values:
    print(value)

[Message(id='msg_p5HOW8V05YyPqJXpxpUvBdoi', assistant_id='asst_1jT3Wgeu5KrPb1QJO062Kh2w', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='{\n    "2024/09/25": "1330.20",\n    "2024/09/26": "1329.40",\n    "주간" : "2024/09/27: 1328.90, 2024/09/28: 1328.10, 2024/09/29: 1327.90, 2024/09/30: 1327.60, 2024/10/01: 1327.30, 2024/10/02: 1327.10, 2024/10/03: 1326.90",\n    "이유" : "2024/09/24와 2024/09/25의 일간 리포트를 분석한 결과, 최근 외환시장 동향과 추세에 따라 앞으로 일주일간 원/달러 환율은 점진적으로 하락할 것으로 예상됩니다. 특별한 외부 충격이 없는 한, 큰 변동 없이 안정적인 추이를 보일 것으로 보입니다."\n}'), type='text')], created_at=1727265553, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_MuIubK2Qac2KibGbDGG5sUSP', status=None, thread_id='thread_Y3ILpCs120aw9mznBehpdV12')]
{
    "2024/09/25": "1330.20",
    "2024/09/26": "1329.40",
    "주간" : "2024/09/27: 1328.90, 2024/09/28: 1328.10, 2024/09/29: 1327.90, 2024/09/30: 1327.60, 2024/10/01: 1327.30, 2024/10/02: 13

In [23]:
from openai import OpenAI
import os

# 환경 변수에서 API 키를 가져옵니다
client = OpenAI(
    api_key="sk-al8LY2KP9JUls1HdCrdrhA2GofI2OztrBR3w1W6pErT3BlbkFJsLFqgNeJKhTgVR3VIBIHVaZ1qvi5FNsS_cBG7rTGIA"
)

# 코드 인터프리터를 tools에 추가합니다
assistant = client.beta.assistants.create(
    name="외환 전문가 어시스턴트",
    instructions="""
        너는 외환 분야 전문가야, 특히 '원/달러' 분야 전문인 경제 전문가야. 
        너에게는 일간, 주간, 월간, 연간 외환 동향을 분석한 리포트가 PDF 파일 형식으로 제공될 거고,
        미국 금리, 유가, 원/달러, DXY의 추세를 기록한 시계열 데이터가 CSV 파일 형식으로 제공될 거야.
        
        먼저 PDF 파일들에 대해서 설명할게:
        1. **Daily240923.pdf**: 2024/09/23의 외환 동향 리포트.
        2. **Daily240924.pdf**: 2024/09/24의 외환 동향 리포트.
        3. **Daily240925.pdf**: 2024/09/25의 외환 동향 리포트.
        4. **Weekly240923.pdf**: 2024/09/23까지의 주간 외환 동향 리포트.
        5. **Monthly2409.pdf**: 2024년 9월 한 달간의 외환 동향 리포트.
        6. **semi2024.pdf**: 2024년 상반기 외환 동향 리포트.

        이 PDF 파일들에 기반해 다음과 같은 분석을 제공해야 해.
        
        다음은 분석 프로세스야 :
        1. PDF를 열람
        2. 결과 값 응답
        
        너는 사용자가 요청한 정보를 항상 다음 양식에 맞춰 반환해야 해(토큰 제한이 있으니까 반드시 응답양식과 일치하는 답변만 해야해):
        
        {
            "2024/09/25": "원/달러 종가",
            "2024/09/26": "원/달러 종가",
            "주간" : "앞으로 한주간 원달러 종가",
            "이유" : "이렇게 분석한 이유"
        }

    """,
    model="gpt-4o",
    tools=[
        {"type": "file_search"},
        {"type": "code_interpreter"}
    ],
)

# 벡터 스토어 생성
vector_store = client.beta.vector_stores.create(name="외환 리포트 벡터 스토어")

# PDF 파일 경로들
pdf_file_paths = [
    "/Users/yang-jin-an/Downloads/report/Daily240923.pdf",
    "/Users/yang-jin-an/Downloads/report/Daily240924.pdf",
    "/Users/yang-jin-an/Downloads/report/Daily240925.pdf",
    "/Users/yang-jin-an/Downloads/report/Weekly240923.pdf",
    "/Users/yang-jin-an/Downloads/report/Monthly2409.pdf",
    "/Users/yang-jin-an/Downloads/report/semi2024.pdf",
]

# PDF 파일 스트림 열기
pdf_file_streams = [open(path, "rb") for path in pdf_file_paths]

# PDF 파일을 업로드하고 벡터 스토어에 추가
pdf_file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=pdf_file_streams
)

# PDF 파일 스트림 닫기
for file_stream in pdf_file_streams:
    file_stream.close()


# 어시스턴트에 새로운 벡터 스토어와 CSV 파일을 추가
assistant = client.beta.assistants.update(
    assistant_id=assistant.id,
    tool_resources={
        "file_search": {"vector_store_ids": [vector_store.id]}
    },
)

# 사용자 메시지로 스레드 생성
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "2024/09/25 2024/09/26 원/달러 환율 종가와 변동범위 그리고 앞으로 일주일간 원/달러 환율 종가를 알려줘 오늘은 2024/09/25 고 원달러 환율을 말할 필요없어 미래에 대한걸 추론해서 말해.",
        }
    ]
)

# 실행 생성 및 완료 대기
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

# 어시스턴트의 응답 가져오기 및 출력
messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))
print(messages)
    
extracted_values = extract_values_from_messages(messages)
for value in extracted_values:
    print(value)

[Message(id='msg_aEB67yfIrXJMzRG4EORmnWNl', assistant_id='asst_Akk5iFc5pBBTRHeDZHn23rBG', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='{\n    "2024/09/25": "1332.10",\n    "2024/09/26": "1329.50",\n    "주간" : "앞으로 한주간 원달러 종가 1326.00~1333.00",\n    "이유" : "최근 며칠간의 환율 동향과 NDF 추세를 기초로 미래의 환율 종가를 추정했습니다. 글로벌 경제 변수와 해외 시장 동향도 고려하여 주간 예측을 제공합니다."\n}'), type='text')], created_at=1727265656, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_bMpvR0d01012nPSSKHuJ4Mh1', status=None, thread_id='thread_wpecmTmxDZXn46d7AC2YXk5b')]
{
    "2024/09/25": "1332.10",
    "2024/09/26": "1329.50",
    "주간" : "앞으로 한주간 원달러 종가 1326.00~1333.00",
    "이유" : "최근 며칠간의 환율 동향과 NDF 추세를 기초로 미래의 환율 종가를 추정했습니다. 글로벌 경제 변수와 해외 시장 동향도 고려하여 주간 예측을 제공합니다."
}


In [24]:
from openai import OpenAI
import os

# 환경 변수에서 API 키를 가져옵니다
client = OpenAI(
    api_key="sk-al8LY2KP9JUls1HdCrdrhA2GofI2OztrBR3w1W6pErT3BlbkFJsLFqgNeJKhTgVR3VIBIHVaZ1qvi5FNsS_cBG7rTGIA"
)

# 코드 인터프리터를 tools에 추가합니다
assistant = client.beta.assistants.create(
    name="외환 전문가 어시스턴트",
    instructions="""
        너는 외환 분야 전문가야, 특히 '원/달러' 분야 전문인 경제 전문가야. 
        너에게는 일간, 주간, 월간, 연간 외환 동향을 분석한 리포트가 PDF 파일 형식으로 제공될 거고,
        미국 금리, 유가, 원/달러, DXY의 추세를 기록한 시계열 데이터가 CSV 파일 형식으로 제공될 거야.
        
        먼저 PDF 파일들에 대해서 설명할게:
        1. **Daily240923.pdf**: 2024/09/23의 외환 동향 리포트.
        2. **Daily240924.pdf**: 2024/09/24의 외환 동향 리포트.
        3. **Daily240925.pdf**: 2024/09/25의 외환 동향 리포트.
        4. **Weekly240923.pdf**: 2024/09/23까지의 주간 외환 동향 리포트.
        5. **Monthly2409.pdf**: 2024년 9월 한 달간의 외환 동향 리포트.
        6. **semi2024.pdf**: 2024년 상반기 외환 동향 리포트.

        이 PDF 파일들에 기반해 다음과 같은 분석을 제공해야 해.
        
        다음은 분석 프로세스야 :
        1. PDF를 열람
        2. 결과 값 응답
        
        너는 사용자가 요청한 정보를 항상 다음 양식에 맞춰 반환해야 해(토큰 제한이 있으니까 반드시 응답양식과 일치하는 답변만 해야해):
        
        {
            "2024/09/25 원/달러 환율": "원/달러 종가",
            "2024/09/25 원/달러 변동폭" "원/달러 변동폭",
            "2024/09/25 이유" : "2024/09/25 원/달러 종가를 이렇게 예측한 이유와 어떤 리포트의 어떤 내용을 기반으로 그렇게 예측했는지 구체적으로 설명",
            "2024/09/26 원/달러 환율": "원/달러 종가",
            "2024/09/26 원/달러 변동폭" "원/달러 변동폭",
            "2024/09/26 이유" : "2024/09/26 원/달러 종가를 이렇게 예측한 이유와 어떤 리포트의 어떤 내용을 기반으로 그렇게 예측했는지 구체적으로 설명",
            "주간" : "앞으로 한주간 원/달러를 하루단위로 종가 반환",
            "주간 이유" : "한주간 원/달러 종가를 이렇게 예측한 이유와 어떤 리포트의 어떤 내용을 기반으로 그렇게 예측했는지 구체적으로 설명"
            "원/달러 추세" : "앞으로 원/달러 추세에 대해 설명하고 왜 그렇게 예측하는지 리포트의 어떤 내용을 기반으로 그렇게 예측했는지 구체적으로 설명"
        }

    """,
    model="gpt-4o",
    tools=[
        {"type": "file_search"},
        {"type": "code_interpreter"}
    ],
)

# 벡터 스토어 생성
vector_store = client.beta.vector_stores.create(name="외환 리포트 벡터 스토어")

# PDF 파일 경로들
pdf_file_paths = [
    "/Users/yang-jin-an/Downloads/report/Daily240923.pdf",
    "/Users/yang-jin-an/Downloads/report/Daily240924.pdf",
    "/Users/yang-jin-an/Downloads/report/Daily240925.pdf",
    "/Users/yang-jin-an/Downloads/report/Weekly240923.pdf",
    "/Users/yang-jin-an/Downloads/report/Monthly2409.pdf",
    "/Users/yang-jin-an/Downloads/report/semi2024.pdf",
]

# PDF 파일 스트림 열기
pdf_file_streams = [open(path, "rb") for path in pdf_file_paths]

# PDF 파일을 업로드하고 벡터 스토어에 추가
pdf_file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=pdf_file_streams
)

# PDF 파일 스트림 닫기
for file_stream in pdf_file_streams:
    file_stream.close()


# 어시스턴트에 새로운 벡터 스토어와 CSV 파일을 추가
assistant = client.beta.assistants.update(
    assistant_id=assistant.id,
    tool_resources={
        "file_search": {"vector_store_ids": [vector_store.id]}
    },
)

# 사용자 메시지로 스레드 생성
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "2024/09/25 2024/09/26 원/달러 환율 종가와 변동범위 그리고 앞으로 일주일간 원/달러 환율 종가를 알려줘 오늘은 2024/09/25 고 원달러 환율을 말할 필요없어 미래에 대한걸 추론해서 말해.",
        }
    ]
)

# 실행 생성 및 완료 대기
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

# 어시스턴트의 응답 가져오기 및 출력
messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))
print(messages)
    
extracted_values = extract_values_from_messages(messages)
for value in extracted_values:
    print(value)

[Message(id='msg_OCU1qdcpgRe0RFIlv5Fhr8xA', assistant_id='asst_JHZNo8eiXcOYvRHjtcF9cUmb', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='{\n    "2024/09/25 원/달러 환율": "1333.90",\n    "2024/09/25 원/달러 변동폭": "1331~1340",\n    "2024/09/25 이유": "9월 24일 리포트 분석을 기반으로 원/달러가 1331~1340 범위에서 변동했음을 알 수 있었습니다. 9월 25일 원/달러 종가는 1333.90입니다.",\n    "2024/09/26 원/달러 환율": "1325.90",\n    "2024/09/26 원/달러 변동폭": "1320~1330",\n    "2024/09/26 이유": "9월 25일 리포트를 기반으로 원/달러가 1320~1330 범위에서 변동했음을 알 수 있었고 NDF 값이 1325.90인 것으로 분석되었습니다.",\n    "주간": {\n        "2024/09/27": "1323.15",\n        "2024/09/28": "1322.00",\n        "2024/09/29": "1320.50",\n        "2024/09/30": "1318.90",\n        "2024/10/01": "1317.90",\n        "2024/10/02": "1316.80",\n        "2024/10/03": "1315.50"\n    },\n    "주간 이유": "주간 리포트를 분석하여 원/달러 환율 변동 추이를 분석을 했다. 특히 미국 금리 정책과 달러화 강세, 세계 경제 동향 등의 변수를 고려했을 때 원/달러 환율이 점진적으로 하락할 것으로 보임. 이와 같은 예측을 기반으로 주간 원/달러 종가를 예측했다.",\n    "원/달러 추세": "앞으로 원/달

In [27]:
# 2024/09/25년 최종
from openai import OpenAI
import os

# 환경 변수에서 API 키를 가져옵니다
client = OpenAI(
    api_key="sk-al8LY2KP9JUls1HdCrdrhA2GofI2OztrBR3w1W6pErT3BlbkFJsLFqgNeJKhTgVR3VIBIHVaZ1qvi5FNsS_cBG7rTGIA"
)

# 코드 인터프리터를 tools에 추가합니다
assistant = client.beta.assistants.create(
    name="외환 전문가 어시스턴트",
    instructions="""
        너는 외환 분야 전문가야, 특히 '원/달러' 분야 전문인 경제 전문가야. 
        너에게는 일간, 주간, 월간, 연간 외환 동향을 분석한 리포트가 PDF 파일 형식으로 제공될 거고,
        미국 금리, 유가, 원/달러, DXY의 추세를 기록한 시계열 데이터가 CSV 파일 형식으로 제공될 거야.
        
        다음은 PDF 파일들에 대한 설명이야:
        1. **Daily240923.pdf**: 2024/09/23의 외환 동향 리포트.
        2. **Daily240924.pdf**: 2024/09/24의 외환 동향 리포트.
        3. **Daily240925.pdf**: 2024/09/25의 외환 동향 리포트.
        4. **Weekly240923.pdf**: 2024/09/23까지의 주간 외환 동향 리포트.
        5. **Monthly2409.pdf**: 2024년 9월 한 달간의 외환 동향 리포트.
        6. **semi2024.pdf**: 2024년 상반기 외환 동향 리포트.
        
        이 PDF 파일들에 기반해 다음과 같은 분석을 제공해야 해.
        
        너는 사용자가 요청한 정보를 항상 다음 양식에 맞춰 반환해야 해. 
        응답은 반드시 **정확하게 이 형식**을 따라야 하며, 만약 이 형식에 맞지 않으면 응답이 잘못된 것으로 간주돼.
        
        ```
        {
            "2024/09/25 원/달러 환율": "<원/달러 종가>",
            "2024/09/25 원/달러 변동폭": "<원/달러 변동폭>",
            "2024/09/25 이유": "<2024/09/25 원/달러 종가를 이렇게 예측한 이유와 어떤 리포트의 어떤 내용을 기반으로 그렇게 예측했는지 구체적으로 설명>",
            "2024/09/26 원/달러 환율": "<원/달러 종가>",
            "2024/09/26 원/달러 변동폭": "<원/달러 변동폭>",
            "2024/09/26 이유": "<2024/09/26 원/달러 종가를 이렇게 예측한 이유와 어떤 리포트의 어떤 내용을 기반으로 그렇게 예측했는지 구체적으로 설명>",
            "주간": {
                "2024/09/27": "<종가>",
                "2024/09/28": "<종가>",
                "2024/09/29": "<종가>",
                "2024/09/30": "<종가>",
                "2024/10/01": "<종가>"
            },
            "주간 이유": "<한주간 원/달러 종가를 이렇게 예측한 이유와 어떤 리포트의 어떤 내용을 기반으로 그렇게 예측했는지 구체적으로 설명>",
            "원/달러 추세": "<앞으로 원/달러 추세에 대해 설명하고 왜 그렇게 예측하는지 리포트의 어떤 내용을 기반으로 그렇게 예측했는지 구체적으로 설명>"
        }
        ```
        
        **중요**: 응답이 반드시 위 형식에 맞아야 하며, 틀릴 경우 잘못된 응답으로 간주돼.
    """,
    model="gpt-4o",
    tools=[
        {"type": "file_search"},
    ],
)

# 벡터 스토어 생성
vector_store = client.beta.vector_stores.create(name="외환 리포트 벡터 스토어")

# PDF 파일 경로들
pdf_file_paths = [
    "/Users/yang-jin-an/Downloads/report/Daily240923.pdf",
    "/Users/yang-jin-an/Downloads/report/Daily240924.pdf",
    "/Users/yang-jin-an/Downloads/report/Daily240925.pdf",
    "/Users/yang-jin-an/Downloads/report/Weekly240923.pdf",
    "/Users/yang-jin-an/Downloads/report/Monthly2409.pdf",
    "/Users/yang-jin-an/Downloads/report/semi2024.pdf",
]

# PDF 파일 스트림 열기
pdf_file_streams = [open(path, "rb") for path in pdf_file_paths]

# PDF 파일을 업로드하고 벡터 스토어에 추가
pdf_file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=pdf_file_streams
)

# PDF 파일 스트림 닫기
for file_stream in pdf_file_streams:
    file_stream.close()


# 어시스턴트에 새로운 벡터 스토어와 CSV 파일을 추가
assistant = client.beta.assistants.update(
    assistant_id=assistant.id,
    tool_resources={
        "file_search": {"vector_store_ids": [vector_store.id]}
    },
)

# 사용자 메시지로 스레드 생성
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "앞으로의 원/달러 환율이 어떻게 될지 알려줘 ",
        }
    ]
)

# 실행 생성 및 완료 대기
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

# 어시스턴트의 응답 가져오기 및 출력
messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))
print(messages)
    
extracted_values = extract_values_from_messages(messages)
for value in extracted_values:
    print(value)

[Message(id='msg_zvAwGzpVMMmqmEDuZMbAlMi0', assistant_id='asst_PSLsbKYUICQODZzKasNW2wZR', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=234, file_citation=FileCitation(file_id='file-luUuO5I8SglweoEiy3VbU8LY'), start_index=212, text='【12:0†Daily240925.pdf】', type='file_citation'), FileCitationAnnotation(end_index=435, file_citation=FileCitation(file_id='file-henoeHMkWzT2uooRXGND1upI'), start_index=413, text='【12:1†Daily240924.pdf】', type='file_citation'), FileCitationAnnotation(end_index=741, file_citation=FileCitation(file_id='file-0lstfOTX0QaUpnUeKojURq4J'), start_index=718, text='【12:3†Weekly240923.pdf】', type='file_citation'), FileCitationAnnotation(end_index=870, file_citation=FileCitation(file_id='file-tZ4go2QrVEbxb4qm4Cfz6ADm'), start_index=851, text='【12:4†semi2024.pdf】', type='file_citation')], value='```\n{\n    "2024/09/25 원/달러 환율": "1330",\n    "2024/09/25 원/달러 변동폭": "10원",\n    "2024/09/25 이유": "2024/09/

In [34]:
from openai import OpenAI
import os

# 환경 변수에서 API 키를 가져옵니다
client = OpenAI(
    api_key="sk-al8LY2KP9JUls1HdCrdrhA2GofI2OztrBR3w1W6pErT3BlbkFJsLFqgNeJKhTgVR3VIBIHVaZ1qvi5FNsS_cBG7rTGIA"
)

# 코드 인터프리터를 tools에 추가합니다
assistant = client.beta.assistants.create(
    name="외환 전문가 어시스턴트",
    instructions="""
        너는 외환 분야 전문가야, 특히 '원/달러' 분야 전문인 경제 전문가야. 
        너에게는 일간, 주간, 월간, 연간 외환 동향을 분석한 리포트가 PDF 파일 형식으로 제공될 거고,
        미국 금리, 유가, 원/달러, DXY의 추세를 기록한 시계열 데이터가 JSON 파일 형식으로 제공될 거야.
        
        먼저 PDF 파일들에 대해서 설명할게:
        1. **Daily240925.pdf**: 2024/09/25의 외환 동향 리포트.
        2. **Weekly240923.pdf**: 2024/09/23까지의 주간 외환 동향 리포트.
        3. **Monthly2409.pdf**: 2024년 9월 한 달간의 외환 동향 리포트.
        4. **semi2024.pdf**: 2024년 상반기 외환 동향 리포트.

        
        다음은 JSON 파일들에 대한 설명이야:
        
        1. **DFII10.json** (2019-2024년 미국 실질 금리 데이터):
            - **date**: 거래 날짜 (YYYY/M/DD 형식).
            - **DFII10**: 해당 날짜의 미국 실질 금리(TIPS) 값. 일부 날짜에는 값이 비어있으며, 이러한 결측치를 제외하고 분석해야함.
        
        2. **dxy.json** (2015-2024년 미국 달러 지수 데이터):
            - **date**: 거래 날짜 (YYYY/M/DD 형식).
            - **end_price**: 장 마감 시 미국 달러 지수 값.
            - **start_price**: 장 시작 시 미국 달러 지수 값.
            - **high_price**: 하루 중 가장 높았던 미국 달러 지수 값.
            - **low_price**: 하루 중 가장 낮았던 미국 달러 지수 값.
            - **fluctuating**: 직전 값과 비교한 변동 비율 (0.01% 단위로 반영). 
        
        3. **WTI_oil.json** (2015-2024년 저유황 경질유(WTI) 유가 데이터):
            - **date**: 거래 날짜 (YYYY/M/DD 형식).
            - **end_price**: 장 마감 
            시 WTI 유가 (1배럴당).
            - **start_price**: 장 시작 시 WTI 유가 (1배럴당).
            - **high_price**: 하루 중 가장 높았던 WTI 유가 (1배럴당).
            - **low_price**: 하루 중 가장 낮았던 WTI 유가 (1배럴당).
            - **trading_amount**: 해당 날짜에 해당하는 거래량. 
            - **fluctuating**: 직전 값과 비교한 변동 비율 (0.01% 단위로 반영).
        
        4. **saudi_oil.json** (2015-2024년 사우디아라비아 유가 데이터):
            - **date**: 거래 날짜 (YYYY/M/DD 형식).
            - **end_price**: 장 마감 시 사우디아라비아 유가 (1배럴당).
            - **start_price**: 장 시작 시 사우디아라비아 유가 (1배럴당).
            - **high_price**: 하루 중 가장 높았던 사우디아라비아 유가 (1배럴당).
            - **low_price**: 하루 중 가장 낮았던 사우디아라비아 유가 (1배럴당).
            - **trading_amount**: 모든값이 null인 데이터 분석필요 없음. 
            - **fluctuating**: 직전 값과 비교한 변동 비율 (0.01% 단위로 반영).
        
        5. **exchange_rate.json** (2010-2024년 원/달러 환율 데이터):
            - **date**: 거래 날짜 (YYYY/M/DD 형식).
            - **base_rate**: 해당 날짜의 매매 기준율 종가.
            
        이 PDF 파일과 JSON 파일에 기반해 다음과 같은 분석을 제공해야 해.
        
        다음은 분석 프로세스야 :
        1. PDF를 열람
        2. JSON를 열람
        3. 결과 값 응답
        
        너는 사용자가 요청한 정보를 항상 다음 양식에 맞춰 반환해야 해(토큰 제한이 있으니까 반드시 응답양식과 일치하는 답변만 해야해):
        
        너는 사용자가 요청한 정보를 항상 다음 양식에 맞춰 반환해야 해. 
        응답은 반드시 **정확하게 이 형식**을 따라야 하며, 만약 이 형식에 맞지 않으면 응답이 잘못된 것으로 간주돼.
        
        ```
        {
            "2024/09/25 원/달러 환율": "<원/달러 종가>",
            "2024/09/25 원/달러 변동 범위": "<원/달러 변동 범위>",
            "2024/09/25 이유": "<2024/09/25 원/달러 종가를 이렇게 예측한 이유와 어떤 리포트와 JSON 파일의 어떤 내용을 기반으로 그렇게 예측했는지 구체적으로 설명>",
            "2024/09/26 원/달러 환율": "<원/달러 종가>",
            "2024/09/26 원/달러 변동 범위": "<원/달러 변동 범위>",
            "2024/09/26 이유": "<2024/09/26 원/달러 종가를 이렇게 예측한 이유와 어떤 리포트와 JSON 파일의 어떤 내용을 기반으로 그렇게 예측했는지 구체적으로 설명>",
            "주간": {
                "2024/09/27": "<종가>",
                "2024/09/28": "<종가>",
                "2024/09/29": "<종가>",
                "2024/09/30": "<종가>",
                "2024/10/01": "<종가>"
            },
            "주간": {
                "2024/09/27": "<종가>",
                "2024/09/28": "<종가>",
                "2024/09/29": "<종가>",
                "2024/09/30": "<종가>",
                "2024/10/01": "<종가>"
            },
            "주간 원달러 변동 범위" : "<주간 원/달러 변동 범위>",
            "주간 이유": "<한주간 원/달러 종가를 이렇게 예측한 이유와 어떤 리포트와 JSON 파일의 어떤 내용을 기반으로 그렇게 예측했는지 구체적으로 설명>",
            "월간": {
                "2024/09/27": "<종가>",
                "2024/09/28": "<종가>",
                "2024/09/29": "<종가>",
                "2024/09/30": "<종가>",
                "2024/10/01": "<종가>",
                "2024/10/02": "<종가>",
                "2024/10/03": "<종가>",
                "2024/10/04": "<종가>",
                "2024/10/05": "<종가>",
                "2024/10/06": "<종가>",
                "2024/10/07": "<종가>",
                "2024/10/08": "<종가>",
                "2024/10/09": "<종가>",
                "2024/10/10": "<종가>",
                "2024/10/11": "<종가>",
                "2024/10/12": "<종가>",
                "2024/10/13": "<종가>",
                "2024/10/14": "<종가>",
                "2024/10/15": "<종가>",
                "2024/10/16": "<종가>",
                "2024/10/17": "<종가>",
                "2024/10/18": "<종가>",
                "2024/10/19": "<종가>",
                "2024/10/20": "<종가>",
                "2024/10/21": "<종가>",
                "2024/10/22": "<종가>",
                "2024/10/23": "<종가>",
                "2024/10/24": "<종가>",
                "2024/10/25": "<종가>",
                "2024/10/26": "<종가>"
            },
            "월간 원달러 변동 범위" : "<월간 원/달러 변동 범위>",
            "월간 이유": "<한달간 원/달러 종가를 이렇게 예측한 이유와 어떤 리포트와 JSON 파일의 내용을 기반으로 그렇게 예측했는지 구체적으로 설명>",
            "최종분석결과": "<앞으로 원/달러 추세에 대한 분석을 말하고 어떤 리포트와 JSON 파일의 내용을 기반으로 그렇게 분석했는지 구체적으로 설명>"
        }
        ```
        
        **중요**: 응답이 반드시 위 형식에 맞아야 하며, 틀릴 경우 잘못된 응답으로 간주돼.
        **중요**: 종가는 소수점 두자리까지 예측해야하여, 범위는 숫자-숫자 양식을 지킬것. 이 양식과 다른 양식으로 응답할 경우 잘못된 응답으로 간주돼.

    """,
    model="gpt-4o-mini",
    tools=[
        {"type": "file_search"}
    ],
)

# 벡터 스토어 생성
vector_store = client.beta.vector_stores.create(name="외환 리포트 벡터 스토어")

# PDF 파일 경로들
pdf_file_paths = [
    "/Users/yang-jin-an/Downloads/report/Daily240925.pdf",
    "/Users/yang-jin-an/Downloads/report/Weekly240923.pdf",
    "/Users/yang-jin-an/Downloads/report/Monthly2409.pdf",
    "/Users/yang-jin-an/Downloads/report/semi2024.pdf",
    "/Users/yang-jin-an/Downloads/rate/dxy.json",
    "/Users/yang-jin-an/Downloads/rate/exchange_rate.json",
    "/Users/yang-jin-an/Downloads/rate/wti_oil.json",
    "/Users/yang-jin-an/Downloads/rate/saudi_oil.json",
    "/Users/yang-jin-an/Downloads/rate/dfii10.json",
    
]

# PDF 파일 스트림 열기
pdf_file_streams = [open(path, "rb") for path in pdf_file_paths]

# PDF 파일을 업로드하고 벡터 스토어에 추가
pdf_file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=pdf_file_streams
)

# PDF 파일 스트림 닫기
for file_stream in pdf_file_streams:
    file_stream.close()

assistant = client.beta.assistants.update(
    assistant_id=assistant.id,
    tool_resources={
        "file_search": {"vector_store_ids": [vector_store.id]}
    },
)

# 사용자 메시지로 스레드 생성
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "2024/09/25 2024/09/26 원/달러 환율 종가와 변동범위 그리고 앞으로 일주일간 원/달러 환율 종가를 알려줘 오늘은 2024/09/25 고 원달러 환율을 말할 필요없어 미래에 대한걸 추론해서 말해.",
        }
    ]
)

# 실행 생성 및 완료 대기
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

# 어시스턴트의 응답 가져오기 및 출력
messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))
print(messages)
    
extracted_values = extract_values_from_messages(messages)
for value in extracted_values:
    print(value)

[Message(id='msg_y851Xli3rT8LPlzLEn9V5t3k', assistant_id='asst_YP6WMe3xSAhfluZPVtOUM3m7', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=290, file_citation=FileCitation(file_id='file-89sHK8Pa6FQAlsduhPIEzCoc'), start_index=278, text='【4:4†source】', type='file_citation'), FileCitationAnnotation(end_index=2132, file_citation=FileCitation(file_id='file-89sHK8Pa6FQAlsduhPIEzCoc'), start_index=2120, text='【4:4†source】', type='file_citation')], value='```\n{\n    "2024/09/25 원/달러 환율": "1,331.80",\n    "2024/09/25 원/달러 변동폭": "1.80",\n    "2024/09/25 이유": "2024년 9월 25일 원/달러 환율은 1,331.80으로 예상되며, 이는 최근의 미국 금리(DFII10)가 감소세를 보이고, DXY 지수가 약간의 상승세를 연출한 데 따른 것으로, 한 보고서에서는 달러화 강세 또는 약세가 꾸준히 이어지는 추세를 반영하고 있습니다. 이러한 추세를 기반으로 원/달러 환율이 예측됩니다 【4:4†source】.",\n    "2024/09/26 원/달러 환율": "1,330.00",\n    "2024/09/26 원/달러 변동폭": "1.80",\n    "2024/09/26 이유": "2024년 9월 26일 원/달러 환율은 1,330.00으로 예상되며, 미국 금리 안정과 더불어 유가 하락세가 긍정적인 영향을 미치고 있습니다. 특히, 

In [35]:
value

'```\n{\n    "2024/09/25 원/달러 환율": "1,331.80",\n    "2024/09/25 원/달러 변동폭": "1.80",\n    "2024/09/25 이유": "2024년 9월 25일 원/달러 환율은 1,331.80으로 예상되며, 이는 최근의 미국 금리(DFII10)가 감소세를 보이고, DXY 지수가 약간의 상승세를 연출한 데 따른 것으로, 한 보고서에서는 달러화 강세 또는 약세가 꾸준히 이어지는 추세를 반영하고 있습니다. 이러한 추세를 기반으로 원/달러 환율이 예측됩니다 【4:4†source】.",\n    "2024/09/26 원/달러 환율": "1,330.00",\n    "2024/09/26 원/달러 변동폭": "1.80",\n    "2024/09/26 이유": "2024년 9월 26일 원/달러 환율은 1,330.00으로 예상되며, 미국 금리 안정과 더불어 유가 하락세가 긍정적인 영향을 미치고 있습니다. 특히, WTI 유가가 하락하는 영향을 받으면서 외환 시장에서도 달러가 약세를 보일 것으로 예상되는 점을 반영합니다  .",\n    "주간": {\n        "2024/09/27": "1,331.50",\n        "2024/09/28": "1,332.00",\n        "2024/09/29": "1,330.50",\n        "2024/09/30": "1,331.00",\n        "2024/10/01": "1,330.50"\n    },\n    "주간 이유": "이번 주(2024년 9월 27일부터 2024년 10월 1일) 원/달러 환율은 1,331.50-1,330.50 사이에서 안정세를 유지할 것으로 보입니다. 이는 DXY 지수가 낮은 변동성을 보이고, 미국 금리의 안정과 원자재 가격 변동성이 낮아진 점과 관련이 있습니다. 이러한 요인들이 결합되어 주간 예측에 기여할 것으로 판단됩니다  .",\n    "월간": {\n        "2024/09/27": "1,331.00",\n      

In [ ]:
# 이거 사용
import json
import mysql.connector
from datetime import datetime

# MySQL 연결 정보 설정
config = {
    'user': 'yangjinan',  # 사용자 이름
    'password': 'ALsk85200!!',  # 비밀번호
    'host': 'hanaex.mysql.database.azure.com',  # 호스트
    'database': 'my_database',  # 데이터베이스 이름
    'port': 3306  # 포트 번호
}

In [37]:
from openai import OpenAI
import os

# 환경 변수에서 API 키를 가져옵니다
client = OpenAI(
    api_key="sk-al8LY2KP9JUls1HdCrdrhA2GofI2OztrBR3w1W6pErT3BlbkFJsLFqgNeJKhTgVR3VIBIHVaZ1qvi5FNsS_cBG7rTGIA"
)

# 코드 인터프리터를 tools에 추가합니다
assistant = client.beta.assistants.create(
    name="외환 전문가 어시스턴트",
    instructions="""
        너는 외환 분야 전문가야, 특히 '원/달러' 분야 전문인 경제 전문가야. 
        너에게는 일간, 주간, 월간, 연간 외환 동향을 분석한 리포트가 PDF 파일 형식으로 제공될 거고,
        미국 금리, 유가, 원/달러, DXY의 추세를 기록한 시계열 데이터가 JSON 파일 형식으로 제공될 거야.
        
        먼저 PDF 파일들에 대해서 설명할게:
        1. **Daily240926.pdf**: 2024/09/26의 외환 동향 리포트.
        2. **Weekly240923.pdf**: 2024/09/23까지의 주간 외환 동향 리포트.
        3. **Monthly2409.pdf**: 2024년 9월 한 달간의 외환 동향 리포트.
        4. **semi2024.pdf**: 2024년 상반기 외환 동향 리포트.

        
        다음은 JSON 파일들에 대한 설명이야:
        
        1. **DFII10.json** (2019-2024년 미국 실질 금리 데이터):
            - **date**: 거래 날짜 (YYYY/M/DD 형식).
            - **DFII10**: 해당 날짜의 미국 실질 금리(TIPS) 값. 일부 날짜에는 값이 비어있으며, 이러한 결측치를 제외하고 분석해야함.
        
        2. **dxy.json** (2015-2024년 미국 달러 지수 데이터):
            - **date**: 거래 날짜 (YYYY/M/DD 형식).
            - **end_price**: 장 마감 시 미국 달러 지수 값.
            - **start_price**: 장 시작 시 미국 달러 지수 값.
            - **high_price**: 하루 중 가장 높았던 미국 달러 지수 값.
            - **low_price**: 하루 중 가장 낮았던 미국 달러 지수 값.
            - **fluctuating**: 직전 값과 비교한 변동 비율 (0.01% 단위로 반영). 
        
        3. **WTI_oil.json** (2015-2024년 저유황 경질유(WTI) 유가 데이터):
            - **date**: 거래 날짜 (YYYY/M/DD 형식).
            - **end_price**: 장 마감 
            시 WTI 유가 (1배럴당).
            - **start_price**: 장 시작 시 WTI 유가 (1배럴당).
            - **high_price**: 하루 중 가장 높았던 WTI 유가 (1배럴당).
            - **low_price**: 하루 중 가장 낮았던 WTI 유가 (1배럴당).
            - **trading_amount**: 해당 날짜에 해당하는 거래량. 
            - **fluctuating**: 직전 값과 비교한 변동 비율 (0.01% 단위로 반영).
        
        4. **saudi_oil.json** (2015-2024년 사우디아라비아 유가 데이터):
            - **date**: 거래 날짜 (YYYY/M/DD 형식).
            - **end_price**: 장 마감 시 사우디아라비아 유가 (1배럴당).
            - **start_price**: 장 시작 시 사우디아라비아 유가 (1배럴당).
            - **high_price**: 하루 중 가장 높았던 사우디아라비아 유가 (1배럴당).
            - **low_price**: 하루 중 가장 낮았던 사우디아라비아 유가 (1배럴당).
            - **trading_amount**: 모든값이 null인 데이터 분석필요 없음. 
            - **fluctuating**: 직전 값과 비교한 변동 비율 (0.01% 단위로 반영).
        
        5. **exchange_rate.json** (2010-2024년 원/달러 환율 데이터):
            - **date**: 거래 날짜 (YYYY/M/DD 형식).
            - **base_rate**: 해당 날짜의 매매 기준율 종가.
            
        이 PDF 파일과 JSON 파일에 기반해 다음과 같은 분석을 제공해야 해.
        
        다음은 분석 프로세스야 :
        1. PDF를 열람
        2. JSON를 열람
        3. 결과 값 응답
        
        너는 사용자가 요청한 정보를 항상 다음 양식에 맞춰 반환해야 해(토큰 제한이 있으니까 반드시 응답양식과 일치하는 답변만 해야해):
        
        너는 사용자가 요청한 정보를 항상 다음 양식에 맞춰 반환해야 해. 
        응답은 반드시 **정확하게 이 형식**을 따라야 하며, 만약 이 형식에 맞지 않으면 응답이 잘못된 것으로 간주돼.
        
        ```
        {
            "2024/09/26 원/달러 환율": "<원/달러 종가>",
            "2024/09/26 원/달러 변동 범위": "<원/달러 변동 범위>",
            "2024/09/26 이유": "<2024/09/26 원/달러 종가를 이렇게 예측한 이유와 어떤 리포트와 JSON 파일의 어떤 내용을 기반으로 그렇게 예측했는지 구체적으로 설명>",
            "2024/09/27 원/달러 환율": "<원/달러 종가>",
            "2024/09/27 원/달러 변동 범위": "<원/달러 변동 범위>",
            "2024/09/27 이유": "<2024/09/27 원/달러 종가를 이렇게 예측한 이유와 어떤 리포트와 JSON 파일의 어떤 내용을 기반으로 그렇게 예측했는지 구체적으로 설명>",
            "주간": {
                "2024/09/28": "<종가>",
                "2024/09/29": "<종가>",
                "2024/09/30": "<종가>",
                "2024/10/01": "<종가>",
                "2024/10/02": "<종가>",
                "2024/10/03": "<종가>",
            },
            "주간 원달러 변동 범위" : "<주간 원/달러 변동 범위>",
            "주간 이유": "<한주간 원/달러 종가를 이렇게 예측한 이유와 어떤 리포트와 JSON 파일의 어떤 내용을 기반으로 그렇게 예측했는지 구체적으로 설명>",
            "월간": {
                "2024/09/28": "<종가>",
                "2024/09/29": "<종가>",
                "2024/09/30": "<종가>",
                "2024/10/01": "<종가>",
                "2024/10/02": "<종가>",
                "2024/10/03": "<종가>",
                "2024/10/04": "<종가>",
                "2024/10/05": "<종가>",
                "2024/10/06": "<종가>",
                "2024/10/07": "<종가>",
                "2024/10/08": "<종가>",
                "2024/10/09": "<종가>",
                "2024/10/10": "<종가>",
                "2024/10/11": "<종가>",
                "2024/10/12": "<종가>",
                "2024/10/13": "<종가>",
                "2024/10/14": "<종가>",
                "2024/10/15": "<종가>",
                "2024/10/16": "<종가>",
                "2024/10/17": "<종가>",
                "2024/10/18": "<종가>",
                "2024/10/19": "<종가>",
                "2024/10/20": "<종가>",
                "2024/10/21": "<종가>",
                "2024/10/22": "<종가>",
                "2024/10/23": "<종가>",
                "2024/10/24": "<종가>",
                "2024/10/25": "<종가>",
                "2024/10/26": "<종가>"
            },
            "월간 원달러 변동 범위" : "<월간 원/달러 변동 범위>",
            "월간 이유": "<한달간 원/달러 종가를 이렇게 예측한 이유와 어떤 리포트와 JSON 파일의 내용을 기반으로 그렇게 예측했는지 구체적으로 설명>",
            "최종분석결과": "<앞으로 원/달러 추세에 대한 분석을 말하고 어떤 리포트와 JSON 파일의 내용을 기반으로 그렇게 분석했는지 구체적으로 설명>"
        }
        ```
        
        **중요**: 응답이 반드시 위 형식에 맞아야 하며, 틀릴 경우 잘못된 응답으로 간주돼.
        **중요**: 종가는 소수점 두자리까지 예측해야하여, 범위는 숫자-숫자 양식을 지킬것. 이 양식과 다른 양식으로 응답할 경우 잘못된 응답으로 간주돼.

    """,
    model="gpt-4o-mini",
    tools=[
        {"type": "file_search"}
    ],
)

# 벡터 스토어 생성
vector_store = client.beta.vector_stores.create(name="외환 리포트 벡터 스토어")

# PDF 파일 경로들
pdf_file_paths = [
    "/Users/yang-jin-an/Downloads/report/Daily240925.pdf",
    "/Users/yang-jin-an/Downloads/report/Weekly240923.pdf",
    "/Users/yang-jin-an/Downloads/report/Monthly2409.pdf",
    "/Users/yang-jin-an/Downloads/report/semi2024.pdf",
    "/Users/yang-jin-an/Downloads/rate/dxy.json",
    "/Users/yang-jin-an/Downloads/rate/exchange_rate.json",
    "/Users/yang-jin-an/Downloads/rate/wti_oil.json",
    "/Users/yang-jin-an/Downloads/rate/saudi_oil.json",
    "/Users/yang-jin-an/Downloads/rate/dfii10.json",
]

# PDF 파일 스트림 열기
pdf_file_streams = [open(path, "rb") for path in pdf_file_paths]

# PDF 파일을 업로드하고 벡터 스토어에 추가
pdf_file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=pdf_file_streams
)

# PDF 파일 스트림 닫기
for file_stream in pdf_file_streams:
    file_stream.close()

assistant = client.beta.assistants.update(
    assistant_id=assistant.id,
    tool_resources={
        "file_search": {"vector_store_ids": [vector_store.id]}
    },
)

# 사용자 메시지로 스레드 생성
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "2024/09/25 2024/09/26 원/달러 환율 종가와 변동범위 그리고 앞으로 일주일간 원/달러 환율 종가를 알려줘 오늘은 2024/09/25 고 원달러 환율을 말할 필요없어 미래에 대한걸 추론해서 말해.",
        }
    ]
)

# 실행 생성 및 완료 대기
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

# 어시스턴트의 응답 가져오기 및 출력
messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))
print(messages)
    
extracted_values = extract_values_from_messages(messages)
for value in extracted_values:
    print(value)

[Message(id='msg_7VRT3Tw75818uirchmej94xR', assistant_id='asst_ijhaWPLgzTK0rje0Cix05Imf', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='```\n{\n    "2024/09/26 원/달러 환율": "1,331.80",\n    "2024/09/26 원/달러 변동 범위": "1,320-1,340",\n    "2024/09/26 이유": "2024년 9월 25일 기준으로 원/달러 환율이 1,331.80에서 마감했습니다. DXY는 100.49로 강세를 보였고, WTI 유가는 71.33으로 다소 하락했습니다. 이러한 요소들은 환율에 영향을 미치며, 특히 유가 하락은 수출 경쟁력을 높이고 원화를 받쳐줄 수 있습니다. 따라서 9월 26일의 원/달러 환율은 1,320에서 1,340 범위 내에서 예상됩니다. 이러한 정보는 Daily240926.pdf와 exchange_rate.json 파일에서 수집되었습니다.",\n    "2024/09/27 원/달러 환율": "1,330.50",\n    "2024/09/27 원/달러 변동 범위": "1,320-1,340",\n    "2024/09/27 이유": "9월 27일에도 DXY의 상승세가 지속되고 있으며, 이는 원화에 부담을 줄 가능성이 높습니다. 월간 리포트와 DXY 데이터 기반으로 원/달러 환율은 1,330.50으로 예측됩니다.",\n    "주간": {\n        "2024/09/28": "1,331.00",\n        "2024/09/29": "1,330.00",\n        "2024/09/30": "1,328.00",\n        "2024/10/01": "1,327.50",\n        "2024/10/02": "1,326.00",\n        "2024/10/03": "1,325.00"\n    }

In [ ]:
from openai import OpenAI
import os

# 환경 변수에서 API 키를 가져옵니다
client = OpenAI(
    api_key=""
)

# 코드 인터프리터를 tools에 추가합니다
assistant = client.beta.assistants.create(
    name="외환 전문가 어시스턴트",
    instructions="""
        너는 외환 분야 전문가야, 특히 '원/달러' 분야 전문인 경제 전문가야. 
        너에게는 일간, 주간, 월간, 연간 외환 동향을 분석한 리포트가 PDF 파일 형식으로 제공될 거고,
        미국 금리, 유가, 원/달러, DXY의 추세를 기록한 시계열 데이터가 JSON 파일 형식으로 제공될 거야.
        
        먼저 PDF 파일들에 대해서 설명할게:
        1. **Daily241010.pdf**: 2024/10/10의 외환 동향 리포트.
        2. **Weekly241007.pdf**: 2024/10/07까지의 주간 외환 동향 리포트.
        3. **Monthly2410.pdf**: 2024년 10월 한 달간의 외환 동향 리포트.
        4. **semi2024.pdf**: 2024년 상반기 외환 동향 리포트.

        
        다음은 JSON 파일들에 대한 설명이야:
        
        1. **DFII10.json** (2019-2024년 미국 실질 금리 데이터):
            - **date**: 거래 날짜 (YYYY/M/DD 형식).
            - **DFII10**: 해당 날짜의 미국 실질 금리(TIPS) 값. 일부 날짜에는 값이 비어있으며, 이러한 결측치를 제외하고 분석해야함.
        
        2. **dxy.json** (2015-2024년 미국 달러 지수 데이터):
            - **date**: 거래 날짜 (YYYY/M/DD 형식).
            - **end_price**: 장 마감 시 미국 달러 지수 값.
            - **start_price**: 장 시작 시 미국 달러 지수 값.
            - **high_price**: 하루 중 가장 높았던 미국 달러 지수 값.
            - **low_price**: 하루 중 가장 낮았던 미국 달러 지수 값.
            - **fluctuating**: 직전 값과 비교한 변동 비율 (0.01% 단위로 반영). 
        
        3. **WTI_oil.json** (2015-2024년 저유황 경질유(WTI) 유가 데이터):
            - **date**: 거래 날짜 (YYYY/M/DD 형식).
            - **end_price**: 장 마감 
            시 WTI 유가 (1배럴당).
            - **start_price**: 장 시작 시 WTI 유가 (1배럴당).
            - **high_price**: 하루 중 가장 높았던 WTI 유가 (1배럴당).
            - **low_price**: 하루 중 가장 낮았던 WTI 유가 (1배럴당).
            - **trading_amount**: 해당 날짜에 해당하는 거래량. 
            - **fluctuating**: 직전 값과 비교한 변동 비율 (0.01% 단위로 반영).
        
        4. **saudi_oil.json** (2015-2024년 사우디아라비아 유가 데이터):
            - **date**: 거래 날짜 (YYYY/M/DD 형식).
            - **end_price**: 장 마감 시 사우디아라비아 유가 (1배럴당).
            - **start_price**: 장 시작 시 사우디아라비아 유가 (1배럴당).
            - **high_price**: 하루 중 가장 높았던 사우디아라비아 유가 (1배럴당).
            - **low_price**: 하루 중 가장 낮았던 사우디아라비아 유가 (1배럴당).
            - **trading_amount**: 모든값이 null인 데이터 분석필요 없음. 
            - **fluctuating**: 직전 값과 비교한 변동 비율 (0.01% 단위로 반영).
        
        5. **exchange_rate.json** (2010-2024년 원/달러 환율 데이터):
            - **date**: 거래 날짜 (YYYY/M/DD 형식).
            - **base_rate**: 해당 날짜의 매매 기준율 종가.
            
        이 PDF 파일과 JSON 파일에 기반해 다음과 같은 분석을 제공해야 해.
        
        다음은 분석 프로세스야 :
        1. PDF를 열람
        2. JSON를 열람
        3. 결과 값 응답
        
        너는 사용자가 요청한 정보를 항상 다음 양식에 맞춰 반환해야 해(토큰 제한이 있으니까 반드시 응답양식과 일치하는 답변만 해야해):
        
        너는 사용자가 요청한 정보를 항상 다음 양식에 맞춰 반환해야 해. 
        응답은 반드시 **정확하게 이 형식**을 따라야 하며, 만약 이 형식에 맞지 않으면 응답이 잘못된 것으로 간주돼.
        
        ```
        {
            "2024/10/10 원/달러 환율": "<원/달러 종가>",
            "2024/10/10 원/달러 변동 범위": "<원/달러 변동 범위>",
            "2024/10/10 이유": "<2024/09/26 원/달러 종가를 이렇게 예측한 이유와 어떤 리포트와 JSON 파일의 어떤 내용을 기반으로 그렇게 예측했는지 구체적으로 설명>",
            "2024/10/11 원/달러 환율": "<원/달러 종가>",
            "2024/10/11 원/달러 변동 범위": "<원/달러 변동 범위>",
            "2024/10/11 이유": "<2024/09/27 원/달러 종가를 이렇게 예측한 이유와 어떤 리포트와 JSON 파일의 어떤 내용을 기반으로 그렇게 예측했는지 구체적으로 설명>",
            "주간": {
                "2024/10/12": "<종가>",
                "2024/10/13": "<종가>",
                "2024/10/14": "<종가>",
                "2024/10/15": "<종가>",
                "2024/10/16": "<종가>",
                "2024/10/17": "<종가>",
            },
            "주간 원달러 변동 범위" : "<주간 원/달러 변동 범위>",
            "주간 이유": "<한주간 원/달러 종가를 이렇게 예측한 이유와 어떤 리포트와 JSON 파일의 어떤 내용을 기반으로 그렇게 예측했는지 구체적으로 설명>",
            "월간": {
                "2024/10/12": "<종가>",
                "2024/10/13": "<종가>",
                "2024/10/14": "<종가>",
                "2024/10/15": "<종가>",
                "2024/10/16": "<종가>",
                "2024/10/17": "<종가>",
                "2024/10/18": "<종가>",
                "2024/10/19": "<종가>",
                "2024/10/20": "<종가>",
                "2024/10/21": "<종가>",
                "2024/10/22": "<종가>",
                "2024/10/23": "<종가>",
                "2024/10/24": "<종가>",
                "2024/10/25": "<종가>",
                "2024/10/26": "<종가>",
                "2024/10/27": "<종가>",
                "2024/10/28": "<종가>",
                "2024/10/29": "<종가>",
                "2024/10/30": "<종가>",
                "2024/10/31": "<종가>",
                "2024/11/01": "<종가>",
                "2024/11/02": "<종가>",
                "2024/11/03": "<종가>",
                "2024/11/04": "<종가>",
                "2024/11/05": "<종가>",
                "2024/11/06": "<종가>",
                "2024/11/07": "<종가>",
                "2024/11/08": "<종가>",
                "2024/11/09": "<종가>"
            },
            "월간 원달러 변동 범위" : "<월간 원/달러 변동 범위>",
            "월간 이유": "<한달간 원/달러 종가를 이렇게 예측한 이유와 어떤 리포트와 JSON 파일의 내용을 기반으로 그렇게 예측했는지 구체적으로 설명>",
            "최종분석결과": "<앞으로 원/달러 추세에 대한 분석을 말하고 어떤 리포트와 JSON 파일의 내용을 기반으로 그렇게 분석했는지 구체적으로 설명>"
        }
        ```
        
        **중요**: 응답이 반드시 위 형식에 맞아야 하며, 틀릴 경우 잘못된 응답으로 간주돼.
        **중요**: 종가는 소수점 두자리까지 예측해야하여, 범위는 숫자-숫자 양식을 지킬것. 이 양식과 다른 양식으로 응답할 경우 잘못된 응답으로 간주돼.

    """,
    model="gpt-4o-mini",
    tools=[
        {"type": "file_search"}
    ],
)

# 벡터 스토어 생성
vector_store = client.beta.vector_stores.create(name="외환 리포트 벡터 스토어")

# PDF 파일 경로들
pdf_file_paths = [
    "/Users/yang-jin-an/Downloads/report/Daily240925.pdf",
    "/Users/yang-jin-an/Downloads/report/Weekly240923.pdf",
    "/Users/yang-jin-an/Downloads/report/Monthly2409.pdf",
    "/Users/yang-jin-an/Downloads/report/semi2024.pdf",
    "/Users/yang-jin-an/Downloads/rate/dxy.json",
    "/Users/yang-jin-an/Downloads/rate/exchange_rate.json",
    "/Users/yang-jin-an/Downloads/rate/wti_oil.json",
    "/Users/yang-jin-an/Downloads/rate/saudi_oil.json",
    "/Users/yang-jin-an/Downloads/rate/dfii10.json",
]

# PDF 파일 스트림 열기
pdf_file_streams = [open(path, "rb") for path in pdf_file_paths]

# PDF 파일을 업로드하고 벡터 스토어에 추가
pdf_file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=pdf_file_streams
)

# PDF 파일 스트림 닫기
for file_stream in pdf_file_streams:
    file_stream.close()

assistant = client.beta.assistants.update(
    assistant_id=assistant.id,
    tool_resources={
        "file_search": {"vector_store_ids": [vector_store.id]}
    },
)

# 사용자 메시지로 스레드 생성
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "2024/09/25 2024/09/26 원/달러 환율 종가와 변동범위 그리고 앞으로 일주일간 원/달러 환율 종가를 알려줘 오늘은 2024/09/25 고 원달러 환율을 말할 필요없어 미래에 대한걸 추론해서 말해.",
        }
    ]
)

# 실행 생성 및 완료 대기
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

# 어시스턴트의 응답 가져오기 및 출력
messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))
print(messages)
    
extracted_values = extract_values_from_messages(messages)
for value in extracted_values:
    print(value)

In [ ]:
from openai import OpenAI
import os

# 환경 변수에서 API 키를 가져옵니다
client = OpenAI(
    api_key=""
)

# 코드 인터프리터를 tools에 추가합니다
assistant = client.beta.assistants.create(
    name="외환 전문가 어시스턴트",
    instructions="""
        너는 외환 분야 전문가야, 특히 '원/달러' 분야 전문인 경제 전문가야. 
        너에게는 일간, 주간, 월간, 연간 외환 동향을 분석한 리포트가 PDF 파일 형식으로 제공될 거고,
        미국 금리, 유가, 원/달러, DXY의 추세를 기록한 시계열 데이터가 JSON 파일 형식으로 제공될 거야.
        
        먼저 PDF 파일들에 대해서 설명할게:
        1. **Daily241010.pdf**: 2024/10/10의 외환 동향 리포트.
        2. **Weekly241007.pdf**: 2024/10/07까지의 주간 외환 동향 리포트.
        3. **Monthly2410.pdf**: 2024년 10월 한 달간의 외환 동향 리포트.
        4. **semi2024.pdf**: 2024년 상반기 외환 동향 리포트.

        
        다음은 JSON 파일들에 대한 설명이야:
        
        1. **DFII10.json** (2019-2024년 미국 실질 금리 데이터):
            - **date**: 거래 날짜 (YYYY/M/DD 형식).
            - **DFII10**: 해당 날짜의 미국 실질 금리(TIPS) 값. 일부 날짜에는 값이 비어있으며, 이러한 결측치를 제외하고 분석해야함.
        
        2. **dxy.json** (2015-2024년 미국 달러 지수 데이터):
            - **date**: 거래 날짜 (YYYY/M/DD 형식).
            - **end_price**: 장 마감 시 미국 달러 지수 값.
            - **start_price**: 장 시작 시 미국 달러 지수 값.
            - **high_price**: 하루 중 가장 높았던 미국 달러 지수 값.
            - **low_price**: 하루 중 가장 낮았던 미국 달러 지수 값.
            - **fluctuating**: 직전 값과 비교한 변동 비율 (0.01% 단위로 반영). 
        
        3. **WTI_oil.json** (2015-2024년 저유황 경질유(WTI) 유가 데이터):
            - **date**: 거래 날짜 (YYYY/M/DD 형식).
            - **end_price**: 장 마감 
            시 WTI 유가 (1배럴당).
            - **start_price**: 장 시작 시 WTI 유가 (1배럴당).
            - **high_price**: 하루 중 가장 높았던 WTI 유가 (1배럴당).
            - **low_price**: 하루 중 가장 낮았던 WTI 유가 (1배럴당).
            - **trading_amount**: 해당 날짜에 해당하는 거래량. 
            - **fluctuating**: 직전 값과 비교한 변동 비율 (0.01% 단위로 반영).
        
        4. **saudi_oil.json** (2015-2024년 사우디아라비아 유가 데이터):
            - **date**: 거래 날짜 (YYYY/M/DD 형식).
            - **end_price**: 장 마감 시 사우디아라비아 유가 (1배럴당).
            - **start_price**: 장 시작 시 사우디아라비아 유가 (1배럴당).
            - **high_price**: 하루 중 가장 높았던 사우디아라비아 유가 (1배럴당).
            - **low_price**: 하루 중 가장 낮았던 사우디아라비아 유가 (1배럴당).
            - **trading_amount**: 모든값이 null인 데이터 분석필요 없음. 
            - **fluctuating**: 직전 값과 비교한 변동 비율 (0.01% 단위로 반영).
        
        5. **exchange_rate.json** (2010-2024년 원/달러 환율 데이터):
            - **date**: 거래 날짜 (YYYY/M/DD 형식).
            - **base_rate**: 해당 날짜의 매매 기준율 종가.
            
        이 PDF 파일과 JSON 파일에 기반해 다음과 같은 분석을 제공해야 해.
        
        다음은 분석 프로세스야 :
        1. PDF를 열람
        2. JSON를 열람
        3. 결과 값 응답
        
        너는 사용자가 요청한 정보를 항상 다음 양식에 맞춰 반환해야 해(토큰 제한이 있으니까 반드시 응답양식과 일치하는 답변만 해야해):
        
        너는 사용자가 요청한 정보를 항상 다음 양식에 맞춰 반환해야 해. 
        응답은 반드시 **정확하게 이 형식**을 따라야 하며, 만약 이 형식에 맞지 않으면 응답이 잘못된 것으로 간주돼.
        
        ```
        {
            "2024/10/10 원/달러 환율": "<원/달러 종가>",
            "2024/10/10 원/달러 변동 범위": "<원/달러 변동 범위>",
            "2024/10/10 이유": "<2024/09/26 원/달러 종가를 이렇게 예측한 이유와 어떤 리포트와 JSON 파일의 어떤 내용을 기반으로 그렇게 예측했는지 구체적으로 설명>",
            "2024/10/11 원/달러 환율": "<원/달러 종가>",
            "2024/10/11 원/달러 변동 범위": "<원/달러 변동 범위>",
            "2024/10/11 이유": "<2024/09/27 원/달러 종가를 이렇게 예측한 이유와 어떤 리포트와 JSON 파일의 어떤 내용을 기반으로 그렇게 예측했는지 구체적으로 설명>",
            "주간": {
                "2024/10/12": "<종가>",
                "2024/10/13": "<종가>",
                "2024/10/14": "<종가>",
                "2024/10/15": "<종가>",
                "2024/10/16": "<종가>",
                "2024/10/17": "<종가>",
            },
            "주간 원달러 변동 범위" : "<주간 원/달러 변동 범위>",
            "주간 이유": "<한주간 원/달러 종가를 이렇게 예측한 이유와 어떤 리포트와 JSON 파일의 어떤 내용을 기반으로 그렇게 예측했는지 구체적으로 설명>",
            "월간": {
                "2024/10/12": "<종가>",
                "2024/10/13": "<종가>",
                "2024/10/14": "<종가>",
                "2024/10/15": "<종가>",
                "2024/10/16": "<종가>",
                "2024/10/17": "<종가>",
                "2024/10/18": "<종가>",
                "2024/10/19": "<종가>",
                "2024/10/20": "<종가>",
                "2024/10/21": "<종가>",
                "2024/10/22": "<종가>",
                "2024/10/23": "<종가>",
                "2024/10/24": "<종가>",
                "2024/10/25": "<종가>",
                "2024/10/26": "<종가>",
                "2024/10/27": "<종가>",
                "2024/10/28": "<종가>",
                "2024/10/29": "<종가>",
                "2024/10/30": "<종가>",
                "2024/10/31": "<종가>",
                "2024/11/01": "<종가>",
                "2024/11/02": "<종가>",
                "2024/11/03": "<종가>",
                "2024/11/04": "<종가>",
                "2024/11/05": "<종가>",
                "2024/11/06": "<종가>",
                "2024/11/07": "<종가>",
                "2024/11/08": "<종가>",
                "2024/11/09": "<종가>"
            },
            "월간 원달러 변동 범위" : "<월간 원/달러 변동 범위>",
            "월간 이유": "<한달간 원/달러 종가를 이렇게 예측한 이유와 어떤 리포트와 JSON 파일의 내용을 기반으로 그렇게 예측했는지 구체적으로 설명>",
            "최종분석결과": "<앞으로 원/달러 추세에 대한 분석을 말하고 어떤 리포트와 JSON 파일의 내용을 기반으로 그렇게 분석했는지 구체적으로 설명>"
        }
        ```
        
        **중요**: 응답이 반드시 위 형식에 맞아야 하며, 틀릴 경우 잘못된 응답으로 간주돼.
        **중요**: 종가는 소수점 두자리까지 예측해야하여, 범위는 숫자-숫자 양식을 지킬것. 이 양식과 다른 양식으로 응답할 경우 잘못된 응답으로 간주돼.

    """,
    model="gpt-4o-mini",
    tools=[
        {"type": "file_search"}
    ],
)

# 벡터 스토어 생성
vector_store = client.beta.vector_stores.create(name="외환 리포트 벡터 스토어")

# PDF 파일 경로들
pdf_file_paths = [
    "/Users/yang-jin-an/Downloads/report/Daily240925.pdf",
    "/Users/yang-jin-an/Downloads/report/Weekly240923.pdf",
    "/Users/yang-jin-an/Downloads/report/Monthly2409.pdf",
    "/Users/yang-jin-an/Downloads/report/semi2024.pdf",
    "/Users/yang-jin-an/Downloads/rate/dxy.json",
    "/Users/yang-jin-an/Downloads/rate/exchange_rate.json",
    "/Users/yang-jin-an/Downloads/rate/wti_oil.json",
    "/Users/yang-jin-an/Downloads/rate/saudi_oil.json",
    "/Users/yang-jin-an/Downloads/rate/dfii10.json",
]

# PDF 파일 스트림 열기
pdf_file_streams = [open(path, "rb") for path in pdf_file_paths]

# PDF 파일을 업로드하고 벡터 스토어에 추가
pdf_file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=pdf_file_streams
)

# PDF 파일 스트림 닫기
for file_stream in pdf_file_streams:
    file_stream.close()

assistant = client.beta.assistants.update(
    assistant_id=assistant.id,
    tool_resources={
        "file_search": {"vector_store_ids": [vector_store.id]}
    },
)

# 사용자 메시지로 스레드 생성
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "2024/09/25 2024/09/26 원/달러 환율 종가와 변동범위 그리고 앞으로 일주일간 원/달러 환율 종가를 알려줘 오늘은 2024/09/25 고 원달러 환율을 말할 필요없어 미래에 대한걸 추론해서 말해.",
        }
    ]
)

# 실행 생성 및 완료 대기
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

# 어시스턴트의 응답 가져오기 및 출력
messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))
print(messages)
    
extracted_values = extract_values_from_messages(messages)
for value in extracted_values:
    print(value)

In [ ]:
{
    "2024/10/10 원/달러 환율": "1347.00",
    "2024/10/10 원/달러 변동 범위": "1341.00-1351.00",
    "2024/10/10 이유": "2024/10/10의 원/달러 환율은 미국 금리 인하에 대한 기대감과 국제 유가 상승으로 인한 원화 약세 요인이 복합적으로 작용하여 1347.00원으로 예측됩니다. 이는 [Daily241010.pdf]와 [Weekly241007.pdf]의 분석을 기반으로 합니다.",
    "2024/10/11 원/달러 환율": "1350.00",
    "2024/10/11 원/달러 변동 범위": "1345.00-1355.00",
    "2024/10/11 이유": "2024/10/10의 원/달러 환율 동향을 바탕으로, 금리 인하 및 유가 상승의 영향으로 환율이 추가 상승할 것으로 보입니다.",
    "주간": {
        "2024/10/12": "1347.00",
        "2024/10/13": "1348.50",
        "2024/10/14": "1350.00",
        "2024/10/15": "1352.00",
        "2024/10/16": "1351.00",
        "2024/10/17": "1354.00"
    },
    "주간 원달러 변동 범위": "1340.00-1355.00",
    "주간 원달러 변동 범위": "1345.00-1365.00",
    "주간 이유": "미국 금리 인하 뉴스와 글로벌 유가 상승 압력으로 인해 원/달러 환율이 지속적으로 상승할 것으로 전망하고 있습니다.`[1]`",
    "월간": {
        "2024/10/12": "1347.50",
        "2024/10/13": "1348.00",
        "2024/10/14": "1350.50",
        "2024/10/15": "1352.50",
        "2024/10/16": "1350.00",
        "2024/10/17": "1354.50",
        "2024/10/18": "1356.00",
        "2024/10/19": "1358.00",
        "2024/10/20": "1357.00",
        "2024/10/21": "1360.00",
        "2024/10/22": "1362.00",
        "2024/10/23": "1364.00",
        "2024/10/24": "1365.00",
        "2024/10/25": "1368.00",
        "2024/10/26": "1370.00",
        "2024/10/27": "1371.00",
        "2024/10/28": "1372.00",
        "2024/10/29": "1375.00",
        "2024/10/30": "1378.00",
        "2024/10/31": "1380.00",
        "2024/11/01": "1382.00",
        "2024/11/02": "1384.00",
        "2024/11/03": "1386.00",
        "2024/11/04": "1388.00",
        "2024/11/05": "1390.00",
        "2024/11/06": "1391.00",
        "2024/11/07": "1392.00",
        "2024/11/08": "1393.00",
        "2024/11/09": "1394.00"
    },
    "월간 원달러 변동 범위": "1340.00-1394.00",
    "월간 이유": "인플레이션 우려와 미국 금리 인하에 대한 기대감으로 인해 원/달러 환율이 지속적으로 상승할 것으로 예상됩니다. 이는 [Monthly2410.pdf]의 분석 내용을 반영합니다.",
    "최종분석결과": "앞으로 원/달러 환율은 미국 금리 인하와 국제 유가 상승의 영향으로 인하여 지속적인 상승 추세를 보일 것으로 예상됩니다. 이는 투자자들에게 주의 깊은 관찰을 요구하는 상황입니다."
}